In [376]:
import numpy as np
import cv2 as cv
import os
import glob
import random
import time
from sklearn.model_selection import train_test_split

In [377]:
# Prepare dataset
DataPath = '/Users/zhuonijie/Desktop/TrainingImages copy/'

def split_data(source, train_dest, test_dest, test_size):
    fileslist = os.listdir(source)
    fileslist_train, fileslist_test = train_test_split(fileslist, test_size=test_size, shuffle=True)

    os.makedirs(train_dest, exist_ok=True)
    os.makedirs(test_dest, exist_ok=True)

    for f in fileslist_train:
        filename = os.path.split(f)[-1]
        os.rename(os.path.join(source, filename), os.path.join(train_dest, filename))
        
    for f in fileslist_test:
        filename = os.path.split(f)[-1]
        os.rename(os.path.join(source, filename), os.path.join(test_dest, filename))

if __name__ == '__main__':
    split_data(DataPath + 'FACES', DataPath + 'train/FACES', DataPath + 'test/FACES', 0.3)
    split_data(DataPath + 'NFACES', DataPath + 'train/NFACES', DataPath + 'test/NFACES', 0.3)

In [379]:
# train and test sets sizes
test_face_image_files = glob.glob(os.path.join(DataPath + 'test/FACES', '*.bmp'), recursive=True)
test_nface_image_files = glob.glob(os.path.join(DataPath + 'test/NFACES', '*.bmp'), recursive=True)
train_face_image_files = glob.glob(os.path.join(DataPath + 'train/FACES', '*.bmp'), recursive=True)
train_nface_image_files = glob.glob(os.path.join(DataPath + 'train/NFACES', '*.bmp'), recursive=True)
print (len(test_face_image_files), len(test_nface_image_files), len(train_face_image_files),len(train_nface_image_files))

1458 2729 3400 6367


In [380]:
# decide width * height of image, select training set size
IMG_SIZE = 24
TRAIN_SIZE_FACE = 2000
TRAIN_SIZE_NFACE = 2000

In [392]:
# load images
def loadImg(fileName,dsize1,dsize2): 
    '''
    load image, resize, rgb2gray, equal histo 
    :param fileName: file name 
    :param dsize: same size, tuple 
    :return: image 
    '''
    img = cv.imread(fileName)
    retImg = cv.resize(img,(dsize1,dsize2)) 
    img = cv.normalize(img, img, 0, 255, cv.NORM_MINMAX)
    retImg = cv.cvtColor(retImg,cv.COLOR_BGR2GRAY) 
    retImg = cv.equalizeHist(retImg)
    return retImg 

In [399]:
# integral images
def integral(img):
    '''
    calculate integral image
    img: the original img array
    itgimg: integral of image
    '''
    h , w = img.shape
    
    itgimg = np.zeros((h+1, w+1), int)
    itgimg[0,0] = img[0,0]
    
    for j in range(1,w):
        itgimg[0,j] = itgimg[0,j-1] + img[0,j]

    for y in range(1, h):
        itgimg[y,0] = itgimg[y-1,0] + img[y,0]
        for x in range(1, w):
            itgimg[y, x] = itgimg[y, x-1] + itgimg[y-1, x] - itgimg[y-1, x-1] + img[y, x]

    return itgimg

In [400]:
# get sum of one rectangular area [x, y, w, h] in one image
def getsum(itgimg, x, y, w, h):
    """Get sum of image in rectangle [x, y, w, h]
    Parameters
    ----------
    itgimg : np.array of shape = [height+1, width+1]
        The integral image.
    x : int
        The starting column index.
    y : int
        The starting row index.
    w : int
        The width of the rectangle.
    h : int
        The height of the rectangle.
    Returns
    -------
    sum : int
    The sum of the pixels in the rectangle, excluding column w and row h.
    """
    x = int(x)
    y = int(y)
    w = int(w)
    h = int(h)
    if x == 0 and y == 0:
        recsum = itgimg[y+h-1, x+w-1]
    elif x == 0:
        recsum = itgimg[y+h-1, x+w-1] - itgimg[y-1, x+w-1]
    elif y == 0:
        recsum = itgimg[y+h-1, x+w-1] - itgimg[y+h-1, x-1]
    else:
        recsum = itgimg[y+h-1, x+w-1] + itgimg[y-1, x-1] - (itgimg[y-1, x+w-1] + itgimg[y+h-1, x-1])
    return recsum

In [401]:
def feature1(itgImage, width, height, stride, increment):
    """ Type 1 filters must have even width.
    Reference points:
        D   E   F
        C   B   A
    Formula: A + C + 2E - 2B - D - F
    """    
    features = []
    for w in range(2 * stride, width, 2 * increment):
        for h in range(stride, height, 1 * increment):
            for x in range(0, (width - w) // stride, stride):
                for y in range(0, (height - h) // stride, stride):
                    A = (x + w , y + h)
                    B = (x + (w // 2) , y + h)
                    C = (x , y + h)
                    D = (x , y)
                    E = (x + (w // 2) , y)
                    F = (x + w , y)
                    
                    white = getsum(itgImage, D[0], D[1], (w // 2), h)
                    black = getsum(itgImage, E[0], E[1], (w // 2), h)
                    score = black - white

                    features.append(score)

    return features

def feature2(itgImage, width, height, stride, increment):
    """ Type 2 filters must have an even height.
    Reference points:
        D   C
        E   B
        F   A
    The sum within upper rectangle: D + B - C - E
    The sum within lower rectangle: E + A - B - F
    Formula: feature2 = upper - lower = 2B + D + F - A - C - 2E
    """
    features = []
    for w in range(stride, width, 1 * increment):
        for h in range(2 * stride, height, 2 * increment):
            for x in range(0, (width - w) // stride, stride):
                for y in range(0, (height - h) // stride, stride):
                    A = (x + w , y + h)
                    B = (x + w , y + (h // 2))
                    C = (x + w , y)
                    D = (x , y)
                    E = (x , y + (h // 2))
                    F = (x , y + h)
                    
                    white = getsum(itgImage, E[0], E[1], w, (h // 2))
                    black = getsum(itgImage, D[0], D[1], w, (h // 2))
                    score = black - white

                    features.append(score)

    return features


def feature3(itgImage, width, height, stride, increment):
    """ Type 3 filters must have a width that is a multiple of 3.
    Reference points:
        E   F   G   H
        D   C   B   A
    Formula: A + 2C + E + 2G - 2B - D - 2F - H
    """
    features = []
    for w in range(3 * stride, width, 3 * increment):
        for h in range(1 * stride, height, 1 * increment):
            for x in range(0, (width - w) // stride, stride):
                for y in range(0, (height - h) // stride, stride):
                    A = (x + w , y + h)
                    B = (x + 2 * (w // 3) , y + h)
                    C = (x + (w // 3) - 1, y + h)
                    D = (x , y + h)
                    E = (x , y)
                    F = (x + (w // 3) , y)
                    G = (x + 2 * (w // 3) , y)
                    H = (x + w , y )

                    white1 = getsum(itgImage, E[0], E[1], (w // 3), h)
                    white2 = getsum(itgImage, G[0], G[1], (w // 3), h)
                    black = getsum(itgImage, F[0], F[1], (w // 3), h)
                    score = black - white1 - white2

                    features.append(score)

    return features

def feature4(itgImage, width, height, stride, increment):
    """ Type 4 filters must have even width and even height.
    Reference points:
        I   H   G
        F   E   D
        C   B   A
    Formula: A + C + 4E + G + I - 2B - 2D - 2F - 2H
    """
    features = []
    for w in range(2 * stride, width, 2 * increment):
        for h in range(2 * stride, height, 2 * increment):
            for x in range(0, (width - w) // stride, stride):
                for y in range(0, (height - h) // stride, stride):
                    A = (x + w , y + h)
                    B = (x + (w // 2) - 1, y + h)
                    C = (x , y + h)
                    D = (x + w , y + (h // 2))
                    E = (x + (w // 2) , y + (h // 2))
                    F = (x , y + (h // 2))
                    G = (x + w , y)
                    H = (x + (w // 2) , y)
                    I = (x , y)

                    white1 = getsum(itgImage, I[0], I[1], (w // 2), (h // 2))
                    white2 = getsum(itgImage, E[0], E[1], (w // 2), (h // 2))
                    black1 = getsum(itgImage, H[0], H[1], (w // 2), (h // 2))
                    black2 = getsum(itgImage, F[0], F[1], (w // 2), (h // 2))
                    score = black1 + black2 - white1 - white2

                    features.append(score)

    return features

In [412]:
def train_classifier(nrounds, featurelists):
    '''
    given a max number of rounds, list of features, and array of features computed over images, create a classifier
    if nrounds = -1, function will continue until the classifier has < 30 percent FPR
    If the classifier hits this before, the function will stop
    '''
    # get number of positive and negative examples
    (nexamples, nfeatures) = featurelists.shape

    npos = TRAIN_SIZE_FACE
    nneg = nexamples - npos
    #nfeatures = len(featurelists[0])
    #nexamples = len(featurelists[0])

    print ("training with " + str(nfeatures) + " features over " + str(nexamples) + " examples")
    
    # create vector of correct classifications
    y = np.ones(nexamples, int)
    y[npos:nexamples] = 0

    # initialize final variables: Features, Indices, Threshold, Parities, Alphas
    final_feature = []
    final_indice = []
    final_threshold = 0
    final_parity = []
    final_alpha = []

    # initialize weights
    weight = np.empty(nexamples)
    weight[0:npos] = 1.0/npos
    weight[npos:nexamples] = 1.0/nneg

    # initialize array for optimal parities and thetas
    parity = np.empty(nfeatures, int)
    theta = np.empty(nfeatures, int)

    # initialize arrays for saving errors
    errors = np.empty((nfeatures, nexamples), int)
    errors_weight = np.empty(nfeatures)
    error_temp = np.empty(nexamples, int)

    # for each round
    cont = True
    r = 0
    while cont:
        start_time = time.time()
        # normalize weights
        weight *= 1.0/sum(weight)
        t_pos = sum(weight*y)
        t_neg = 1-t_pos

        print ("\tround " + str(r+1) + ":")
        print (len(featurelists[:,1]))

        # for each feature
        for f in range(0,nfeatures):
            # initialize positive, negative to the left total weight
            s_pos = 0
            s_neg = 0
            # set weighted error to an impossibly high value
            errors_weight[f] = 2
            # get permutation that sorts the computed features
            fsort = np.argsort(featurelists[:,f])
            #print (len(fsort))
            
            # for each spot between examples
            for n in range(0,len(featurelists[:,f])):
                # n is the sorted index
                # i is index of the corresponding example
                i = fsort[n]
                
                # increment weights to the left
                if y[i]:
                    s_pos += weight[i]
                else:
                    s_neg += weight[i]
                
                # find weighted errors
                e_pos = s_pos + t_neg - s_neg
                e_neg = s_neg + t_pos - s_pos

                # store best threshold, parity
                if e_neg < errors_weight[f]:
                    errors_weight[f] = e_neg
                    parity[f] = -1
                    if n+1 >= len(fsort):
                        theta[f] = featurelists[i,f]
                    else:
                        theta[f] = (featurelists[i,f] + featurelists[fsort[n+1],f])/2
                if e_pos < errors_weight[f]:
                    errors_weight[f] = e_pos
                    parity[f] = 1
                    if n+1 >= len(fsort):
                        theta[f] = featurelists[i,f]
                    else:
                        theta[f] = (featurelists[i,f] + featurelists[fsort[n+1],f])/2
                        
            # compute the actual errors
            errors[f,0:npos] = (parity[f]*featurelists[0:npos,f]) < parity[f]*theta[f]
            errors[f,npos:nexamples] = (parity[f]*featurelists[npos:nexamples,f]) > parity[f]*theta[f]
            errors_weight[f] = sum(errors[f,:]*weight)

        # get best feature
        best = np.argmin(errors_weight)
        best_error = errors_weight[best]
        
        # save best feature and associated data to respective lists
        final_indice.append(best)
        final_feature.append(featurelists[:,best])
        final_parity.append(parity[best])
        # get beta value
        # this if statement is just a redundancy in case of perfect classifiers
        if best_error == 0:
            beta = 0.1
            # store alpha value
            final_alpha.append(1)
        else:
            beta = best_error/(1-best_error)
            # store alpha value
            final_alpha.append(np.log(1.0/beta))
        # update weights
        for n in range(0,nexamples):
            weight[n] *= beta**(1-errors[best, n])


        end_time = time.time()
        print ("\tdone in " + str(end_time-start_time) + " seconds")
        print ("\tbest classifier number " + str(best) + " with " + str(sum(errors[best,:])) + " errors and weighed error " + str(best_error))
        print ("\tparity " + str(parity[best]))
        
        # get evaluation of final classifier so far
        feature_temp = np.zeros(nexamples)
        # compute weighted features for classifier for each image
        for s in range(0,r):
            i = final_indice[s]
            feature_temp += final_alpha[s]*final_parity[s]*featurelists[:,i]
        # get final threshold
        final_threshold = min(feature_temp[0:npos])
        
        # find errors
        error_temp[0:npos] = feature_temp[0:npos] < final_threshold
        error_temp[npos:nexamples] = feature_temp[npos:nexamples] >= final_threshold
        
        # get number of false positives
        err = sum(error_temp)
        print ("\tfalse positives so far: " + str(err) + " or " + str(float(err)/nneg) + "\n")
        # increment r, continuity variable
        
        r += 1
        if (float(err)/nneg) < 0.3:
            break
        if nrounds != -1:
            cont = r < nrounds
            
    # output = the items to keep in the training set
    output = np.logical_not(error_temp)
    return (final_feature, final_alpha, final_parity, final_threshold, final_indice, output)

In [420]:
# classifier cascade
def cascade(nstages, nrounds, featurelists, images):
    '''
    given a max number of stages, max number of rounds, vector of features, and array of integral images,
    create a cascade of classifiers
    if nstages = -1, function will continue until cascade hits <1 percent FPR
    If the cascade hits this before, the function will stop.
    '''
    # initialize final variables
    features = []
    alphas = []
    parities = []
    thresholds = []
    indices = []

    # stage counter and continuity variable
    n = 0
    cont = True
    # for each stage
    while cont:
        print ("stage " + str(n) + ":")
        # calculate results of feautres for all training images

        # do adaboost/one stage
        (final_f, final_a, final_p, final_t, final_i, output) = train_classifier(nrounds, featurelists)

        # append classifier info to final variables
        features.append(final_f)
        alphas.append(final_a)
        parities.append(final_p)
        thresholds.append(final_t)
        indices.append(final_i)

        # remove easily classified examples
        output[TRAIN_SIZE_FACE:] = output[TRAIN_SIZE_FACE:] == 0
        featurelists = featurelists[output[:] == 1,:]
        #images = images[output[:] == 1,:,:]
        #print ('hi')

        # remove used features from the list of features to avoid cycles
        ftrs = np.delete(featurelists, final_i, axis=1)
        
        n += 1
        if nstages != -1:
            cont = n < nstages
    return (features, alphas, parities, thresholds, indices)

In [454]:
def evaluate(fc, fa, fp, ft, fi):
    '''
    given an integral image and a classifier, evaluate whether the image is a face
    '''
    total = 0
    for i in range(0,len(fa)):
        total += fa[i]*fc[fi[i]]*fp[i]
    return total >= ft

def evaluate_cascade(fc, fa, fp, ft, fi):
    '''
    given a cascade, evaluate whether an integral image is a face
    '''
    n = len(fa)
    for i in range(0,n):
        x = evaluate(fc, fa[i], fp[i], ft[i], fi[i])
        if (not x):
            return 0
    return 1

In [397]:
# randomly select an image and have a look
random.seed(1000)
random_face_files = random.sample(train_face_image_files, 5)
len(loadImg(train_face_image_files[0],IMG_SIZE,IMG_SIZE))

24

In [398]:
# import images
image = np.empty((IMG_SIZE, IMG_SIZE), int)
images = []
for i in range(0, TRAIN_SIZE_FACE):
    images.append(loadImg(train_face_image_files[i],IMG_SIZE,IMG_SIZE))
for i in range(0, TRAIN_SIZE_NFACE):
    images.append(loadImg(train_nface_image_files[i],IMG_SIZE,IMG_SIZE))

In [405]:
# make integral image
iimages = []
for i in range(len(images)):
    iimages.append(integral(images[i]))

In [410]:
stride = 2
increment = 2
features = []
for i in range(len(images)):
    ftrs1 = feature1(iimages[i], IMG_SIZE, IMG_SIZE, stride, increment)
    ftrs2 = feature2(iimages[i], IMG_SIZE, IMG_SIZE, stride, increment)
    ftrs3 = feature3(iimages[i], IMG_SIZE, IMG_SIZE, stride, increment)
    ftrs4 = feature4(iimages[i], IMG_SIZE, IMG_SIZE, stride, increment)
    features.append(ftrs1 + ftrs2 + ftrs3 + ftrs4)

In [513]:
featurelists = np.array(features)
(fc, fa, fp, ft, fi) = cascade(1, 80, featurelists, imagesv)

stage 0:
training with 1665 features over 4000 examples
	round 1:
4000
	done in 7.731911897659302 seconds
	best classifier number 1072 with 477 errors and weighed error 0.11925000000000845
	parity -1
	false positives so far: 2000 or 1.0

	round 2:
4000
	done in 7.2774977684021 seconds
	best classifier number 132 with 648 errors and weighed error 0.18803418803419314
	parity 1
	false positives so far: 1659 or 0.8295

	round 3:
4000
	done in 7.261751174926758 seconds
	best classifier number 1129 with 688 errors and weighed error 0.24671417820974234
	parity -1
	false positives so far: 1652 or 0.826

	round 4:
4000
	done in 7.054962158203125 seconds
	best classifier number 1644 with 1129 errors and weighed error 0.28193276508842147
	parity 1
	false positives so far: 1655 or 0.8275

	round 5:
4000
	done in 6.862055063247681 seconds
	best classifier number 955 with 888 errors and weighed error 0.30335224834872365
	parity 1
	false positives so far: 1369 or 0.6845

	round 6:
4000
	done in 7.241

	done in 6.606097936630249 seconds
	best classifier number 1069 with 1308 errors and weighed error 0.3942172263785378
	parity -1
	false positives so far: 1284 or 0.642

	round 47:
4000
	done in 6.701181888580322 seconds
	best classifier number 29 with 1259 errors and weighed error 0.3893137967136859
	parity 1
	false positives so far: 1315 or 0.6575

	round 48:
4000
	done in 6.724599838256836 seconds
	best classifier number 0 with 1449 errors and weighed error 0.3895490428533239
	parity 1
	false positives so far: 1309 or 0.6545

	round 49:
4000
	done in 6.639173269271851 seconds
	best classifier number 808 with 2637 errors and weighed error 0.3778535549965072
	parity -1
	false positives so far: 1306 or 0.653

	round 50:
4000
	done in 6.724200963973999 seconds
	best classifier number 1570 with 970 errors and weighed error 0.39574040907816677
	parity 1
	false positives so far: 1307 or 0.6535

	round 51:
4000
	done in 6.692472219467163 seconds
	best classifier number 590 with 1444 errors a

In [514]:
test_face_image_files = glob.glob(os.path.join(DataPath + 'test/FACES', '*.bmp'), recursive=True)

evaluation1 = 0

for i in range(1000):
    # import testing image
    bigimg = loadImg(test_face_image_files[i],IMG_SIZE,IMG_SIZE)

    iimg = integral(bigimg)

    stride = 2 
    increment = 2

    ftrs1 = feature1(iimg, IMG_SIZE, IMG_SIZE, stride, increment)
    ftrs2 = feature2(iimg, IMG_SIZE, IMG_SIZE, stride, increment)
    ftrs3 = feature3(iimg, IMG_SIZE, IMG_SIZE, stride, increment)
    ftrs4 = feature4(iimg, IMG_SIZE, IMG_SIZE, stride, increment)

    fts = ftrs1 + ftrs2 + ftrs3 + ftrs4 

    # evaluate whether there are faces in image
    evaluation1 += evaluate_cascade(fts, fa, fp, ft, fi)

print (evaluation1/1000)

0.999


In [515]:
test_nface_image_files = glob.glob(os.path.join(DataPath + 'test/NFACES', '*.bmp'), recursive=True)

evaluation2 = 0

for i in range(1000):
    # import testing image
    bigimg = loadImg(test_nface_image_files[i],IMG_SIZE,IMG_SIZE)

    iimg = integral(bigimg)

    stride = 2 
    increment = 2

    ftrs1 = feature1(iimg, IMG_SIZE, IMG_SIZE, stride, increment)
    ftrs2 = feature2(iimg, IMG_SIZE, IMG_SIZE, stride, increment)
    ftrs3 = feature3(iimg, IMG_SIZE, IMG_SIZE, stride, increment)
    ftrs4 = feature4(iimg, IMG_SIZE, IMG_SIZE, stride, increment)

    fts = ftrs1 + ftrs2 + ftrs3 + ftrs4 

    # evaluate whether there are faces in image
    evaluation2 += evaluate_cascade(fts, fa, fp, ft, fi)

print (evaluation2/1000)

0.696


In [516]:
featurelists = np.array(features)
(fc, fa, fp, ft, fi) = cascade(2, 80, featurelists, imagesv)

stage 0:
training with 1665 features over 4000 examples
	round 1:
4000
	done in 7.895150899887085 seconds
	best classifier number 1072 with 477 errors and weighed error 0.11925000000000845
	parity -1
	false positives so far: 2000 or 1.0

	round 2:
4000
	done in 7.386625051498413 seconds
	best classifier number 132 with 648 errors and weighed error 0.18803418803419314
	parity 1
	false positives so far: 1659 or 0.8295

	round 3:
4000
	done in 7.301009178161621 seconds
	best classifier number 1129 with 688 errors and weighed error 0.24671417820974234
	parity -1
	false positives so far: 1652 or 0.826

	round 4:
4000
	done in 6.894897937774658 seconds
	best classifier number 1644 with 1129 errors and weighed error 0.28193276508842147
	parity 1
	false positives so far: 1655 or 0.8275

	round 5:
4000
	done in 6.927364110946655 seconds
	best classifier number 955 with 888 errors and weighed error 0.30335224834872365
	parity 1
	false positives so far: 1369 or 0.6845

	round 6:
4000
	done in 7.3

	done in 6.538229942321777 seconds
	best classifier number 1069 with 1308 errors and weighed error 0.3942172263785378
	parity -1
	false positives so far: 1284 or 0.642

	round 47:
4000
	done in 6.722947835922241 seconds
	best classifier number 29 with 1259 errors and weighed error 0.3893137967136859
	parity 1
	false positives so far: 1315 or 0.6575

	round 48:
4000
	done in 6.760776996612549 seconds
	best classifier number 0 with 1449 errors and weighed error 0.3895490428533239
	parity 1
	false positives so far: 1309 or 0.6545

	round 49:
4000
	done in 6.6916468143463135 seconds
	best classifier number 808 with 2637 errors and weighed error 0.3778535549965072
	parity -1
	false positives so far: 1306 or 0.653

	round 50:
4000
	done in 6.892768859863281 seconds
	best classifier number 1570 with 970 errors and weighed error 0.39574040907816677
	parity 1
	false positives so far: 1307 or 0.6535

	round 51:
4000
	done in 6.673426866531372 seconds
	best classifier number 590 with 1444 errors 

	done in 5.522050142288208 seconds
	best classifier number 1376 with 787 errors and weighed error 0.3508620718450178
	parity 1
	false positives so far: 1025 or 0.73005698005698

	round 11:
3404
	done in 5.578579902648926 seconds
	best classifier number 267 with 994 errors and weighed error 0.3625066354568021
	parity -1
	false positives so far: 1077 or 0.7670940170940171

	round 12:
3404
	done in 5.472288131713867 seconds
	best classifier number 640 with 2024 errors and weighed error 0.3547309807175117
	parity -1
	false positives so far: 977 or 0.6958689458689459

	round 13:
3404
	done in 5.590029954910278 seconds
	best classifier number 1236 with 852 errors and weighed error 0.36925835785109073
	parity -1
	false positives so far: 936 or 0.6666666666666666

	round 14:
3404
	done in 5.595811128616333 seconds
	best classifier number 1072 with 567 errors and weighed error 0.374416085002934
	parity -1
	false positives so far: 963 or 0.6858974358974359

	round 15:
3404
	done in 5.45717287063

	done in 5.425506114959717 seconds
	best classifier number 1126 with 1242 errors and weighed error 0.38628601471726126
	parity 1
	false positives so far: 1167 or 0.8311965811965812

	round 53:
3404
	done in 5.384148836135864 seconds
	best classifier number 1073 with 1315 errors and weighed error 0.39280646682584325
	parity -1
	false positives so far: 1166 or 0.8304843304843305

	round 54:
3404
	done in 5.4353978633880615 seconds
	best classifier number 984 with 1833 errors and weighed error 0.38009023380610796
	parity 1
	false positives so far: 1143 or 0.8141025641025641

	round 55:
3404
	done in 5.493680953979492 seconds
	best classifier number 1307 with 1418 errors and weighed error 0.40764100853243873
	parity -1
	false positives so far: 1112 or 0.792022792022792

	round 56:
3404
	done in 5.4620091915130615 seconds
	best classifier number 728 with 903 errors and weighed error 0.39663594125196744
	parity -1
	false positives so far: 1130 or 0.8048433048433048

	round 57:
3404
	done in 

In [517]:
test_face_image_files = glob.glob(os.path.join(DataPath + 'test/FACES', '*.bmp'), recursive=True)

evaluation1 = 0

for i in range(1000):
    # import testing image
    bigimg = loadImg(test_face_image_files[i],IMG_SIZE,IMG_SIZE)

    iimg = integral(bigimg)

    stride = 2 
    increment = 2

    ftrs1 = feature1(iimg, IMG_SIZE, IMG_SIZE, stride, increment)
    ftrs2 = feature2(iimg, IMG_SIZE, IMG_SIZE, stride, increment)
    ftrs3 = feature3(iimg, IMG_SIZE, IMG_SIZE, stride, increment)
    ftrs4 = feature4(iimg, IMG_SIZE, IMG_SIZE, stride, increment)

    fts = ftrs1 + ftrs2 + ftrs3 + ftrs4 

    # evaluate whether there are faces in image
    evaluation1 += evaluate_cascade(fts, fa, fp, ft, fi)

print (evaluation1/1000)

0.999


In [518]:
test_nface_image_files = glob.glob(os.path.join(DataPath + 'test/NFACES', '*.bmp'), recursive=True)

evaluation2 = 0

for i in range(1000):
    # import testing image
    bigimg = loadImg(test_nface_image_files[i],IMG_SIZE,IMG_SIZE)

    iimg = integral(bigimg)

    stride = 2 
    increment = 2

    ftrs1 = feature1(iimg, IMG_SIZE, IMG_SIZE, stride, increment)
    ftrs2 = feature2(iimg, IMG_SIZE, IMG_SIZE, stride, increment)
    ftrs3 = feature3(iimg, IMG_SIZE, IMG_SIZE, stride, increment)
    ftrs4 = feature4(iimg, IMG_SIZE, IMG_SIZE, stride, increment)

    fts = ftrs1 + ftrs2 + ftrs3 + ftrs4 

    # evaluate whether there are faces in image
    evaluation2 += evaluate_cascade(fts, fa, fp, ft, fi)

print (evaluation2/1000)

0.564


In [519]:
featurelists = np.array(features)
(fc, fa, fp, ft, fi) = cascade(3, 80, featurelists, imagesv)

stage 0:
training with 1665 features over 4000 examples
	round 1:
4000
	done in 7.627806186676025 seconds
	best classifier number 1072 with 477 errors and weighed error 0.11925000000000845
	parity -1
	false positives so far: 2000 or 1.0

	round 2:
4000
	done in 7.114388942718506 seconds
	best classifier number 132 with 648 errors and weighed error 0.18803418803419314
	parity 1
	false positives so far: 1659 or 0.8295

	round 3:
4000
	done in 6.9561378955841064 seconds
	best classifier number 1129 with 688 errors and weighed error 0.24671417820974234
	parity -1
	false positives so far: 1652 or 0.826

	round 4:
4000
	done in 6.777761936187744 seconds
	best classifier number 1644 with 1129 errors and weighed error 0.28193276508842147
	parity 1
	false positives so far: 1655 or 0.8275

	round 5:
4000
	done in 6.7790210247039795 seconds
	best classifier number 955 with 888 errors and weighed error 0.30335224834872365
	parity 1
	false positives so far: 1369 or 0.6845

	round 6:
4000
	done in 6

	done in 6.607101917266846 seconds
	best classifier number 1069 with 1308 errors and weighed error 0.3942172263785378
	parity -1
	false positives so far: 1284 or 0.642

	round 47:
4000
	done in 6.636744260787964 seconds
	best classifier number 29 with 1259 errors and weighed error 0.3893137967136859
	parity 1
	false positives so far: 1315 or 0.6575

	round 48:
4000
	done in 6.729403972625732 seconds
	best classifier number 0 with 1449 errors and weighed error 0.3895490428533239
	parity 1
	false positives so far: 1309 or 0.6545

	round 49:
4000
	done in 6.606115102767944 seconds
	best classifier number 808 with 2637 errors and weighed error 0.3778535549965072
	parity -1
	false positives so far: 1306 or 0.653

	round 50:
4000
	done in 6.749648094177246 seconds
	best classifier number 1570 with 970 errors and weighed error 0.39574040907816677
	parity 1
	false positives so far: 1307 or 0.6535

	round 51:
4000
	done in 6.666522979736328 seconds
	best classifier number 590 with 1444 errors a

	done in 5.404282808303833 seconds
	best classifier number 1376 with 787 errors and weighed error 0.3508620718450178
	parity 1
	false positives so far: 1025 or 0.73005698005698

	round 11:
3404
	done in 5.438683032989502 seconds
	best classifier number 267 with 994 errors and weighed error 0.3625066354568021
	parity -1
	false positives so far: 1077 or 0.7670940170940171

	round 12:
3404
	done in 5.411592960357666 seconds
	best classifier number 640 with 2024 errors and weighed error 0.3547309807175117
	parity -1
	false positives so far: 977 or 0.6958689458689459

	round 13:
3404
	done in 5.538711071014404 seconds
	best classifier number 1236 with 852 errors and weighed error 0.36925835785109073
	parity -1
	false positives so far: 936 or 0.6666666666666666

	round 14:
3404
	done in 5.542057991027832 seconds
	best classifier number 1072 with 567 errors and weighed error 0.374416085002934
	parity -1
	false positives so far: 963 or 0.6858974358974359

	round 15:
3404
	done in 5.42600011825

	done in 5.565013885498047 seconds
	best classifier number 1126 with 1242 errors and weighed error 0.38628601471726126
	parity 1
	false positives so far: 1167 or 0.8311965811965812

	round 53:
3404
	done in 5.516163110733032 seconds
	best classifier number 1073 with 1315 errors and weighed error 0.39280646682584325
	parity -1
	false positives so far: 1166 or 0.8304843304843305

	round 54:
3404
	done in 5.691342830657959 seconds
	best classifier number 984 with 1833 errors and weighed error 0.38009023380610796
	parity 1
	false positives so far: 1143 or 0.8141025641025641

	round 55:
3404
	done in 5.710811138153076 seconds
	best classifier number 1307 with 1418 errors and weighed error 0.40764100853243873
	parity -1
	false positives so far: 1112 or 0.792022792022792

	round 56:
3404
	done in 5.559200048446655 seconds
	best classifier number 728 with 903 errors and weighed error 0.39663594125196744
	parity -1
	false positives so far: 1130 or 0.8048433048433048

	round 57:
3404
	done in 5.

	done in 5.201837062835693 seconds
	best classifier number 1078 with 1894 errors and weighed error 0.35179473703570124
	parity 1
	false positives so far: 942 or 0.8328912466843501

	round 15:
3131
	done in 5.2882771492004395 seconds
	best classifier number 1067 with 1636 errors and weighed error 0.3495057661242725
	parity -1
	false positives so far: 994 or 0.878868258178603

	round 16:
3131
	done in 5.24650502204895 seconds
	best classifier number 135 with 1353 errors and weighed error 0.3676296468162115
	parity 1
	false positives so far: 1018 or 0.9000884173297966

	round 17:
3131
	done in 5.360038995742798 seconds
	best classifier number 96 with 563 errors and weighed error 0.3599458724864811
	parity 1
	false positives so far: 1037 or 0.9168877099911583

	round 18:
3131
	done in 5.359555721282959 seconds
	best classifier number 1073 with 1114 errors and weighed error 0.372468597638847
	parity -1
	false positives so far: 1034 or 0.9142351900972591

	round 19:
3131
	done in 5.354974746

	done in 5.2666380405426025 seconds
	best classifier number 1073 with 1295 errors and weighed error 0.3977592432168578
	parity -1
	false positives so far: 985 or 0.8709106984969054

	round 57:
3131
	done in 5.0887720584869385 seconds
	best classifier number 1102 with 806 errors and weighed error 0.39483591372665694
	parity -1
	false positives so far: 952 or 0.8417329796640142

	round 58:
3131
	done in 5.255703926086426 seconds
	best classifier number 1257 with 1619 errors and weighed error 0.3850828332687182
	parity 1
	false positives so far: 948 or 0.8381962864721485

	round 59:
3131
	done in 5.237345933914185 seconds
	best classifier number 1551 with 1139 errors and weighed error 0.39559262737973266
	parity 1
	false positives so far: 891 or 0.7877984084880637

	round 60:
3131
	done in 5.241918087005615 seconds
	best classifier number 1365 with 1614 errors and weighed error 0.38811932934630605
	parity -1
	false positives so far: 893 or 0.7895667550839964

	round 61:
3131
	done in 5.34

In [520]:
test_face_image_files = glob.glob(os.path.join(DataPath + 'test/FACES', '*.bmp'), recursive=True)

evaluation1 = 0

for i in range(1000):
    # import testing image
    bigimg = loadImg(test_face_image_files[i],IMG_SIZE,IMG_SIZE)

    iimg = integral(bigimg)

    stride = 2 
    increment = 2

    ftrs1 = feature1(iimg, IMG_SIZE, IMG_SIZE, stride, increment)
    ftrs2 = feature2(iimg, IMG_SIZE, IMG_SIZE, stride, increment)
    ftrs3 = feature3(iimg, IMG_SIZE, IMG_SIZE, stride, increment)
    ftrs4 = feature4(iimg, IMG_SIZE, IMG_SIZE, stride, increment)

    fts = ftrs1 + ftrs2 + ftrs3 + ftrs4 

    # evaluate whether there are faces in image
    evaluation1 += evaluate_cascade(fts, fa, fp, ft, fi)

print (evaluation1/1000)

0.999


In [521]:
test_nface_image_files = glob.glob(os.path.join(DataPath + 'test/NFACES', '*.bmp'), recursive=True)

evaluation2 = 0

for i in range(1000):
    # import testing image
    bigimg = loadImg(test_nface_image_files[i],IMG_SIZE,IMG_SIZE)

    iimg = integral(bigimg)

    stride = 2 
    increment = 2

    ftrs1 = feature1(iimg, IMG_SIZE, IMG_SIZE, stride, increment)
    ftrs2 = feature2(iimg, IMG_SIZE, IMG_SIZE, stride, increment)
    ftrs3 = feature3(iimg, IMG_SIZE, IMG_SIZE, stride, increment)
    ftrs4 = feature4(iimg, IMG_SIZE, IMG_SIZE, stride, increment)

    fts = ftrs1 + ftrs2 + ftrs3 + ftrs4 

    # evaluate whether there are faces in image
    evaluation2 += evaluate_cascade(fts, fa, fp, ft, fi)

print (evaluation2/1000)

0.524


In [522]:
featurelists = np.array(features)
(fc, fa, fp, ft, fi) = cascade(4, 80, featurelists, imagesv)

stage 0:
training with 1665 features over 4000 examples
	round 1:
4000
	done in 7.508281230926514 seconds
	best classifier number 1072 with 477 errors and weighed error 0.11925000000000845
	parity -1
	false positives so far: 2000 or 1.0

	round 2:
4000
	done in 6.871346950531006 seconds
	best classifier number 132 with 648 errors and weighed error 0.18803418803419314
	parity 1
	false positives so far: 1659 or 0.8295

	round 3:
4000
	done in 6.909687042236328 seconds
	best classifier number 1129 with 688 errors and weighed error 0.24671417820974234
	parity -1
	false positives so far: 1652 or 0.826

	round 4:
4000
	done in 6.69916296005249 seconds
	best classifier number 1644 with 1129 errors and weighed error 0.28193276508842147
	parity 1
	false positives so far: 1655 or 0.8275

	round 5:
4000
	done in 6.945468902587891 seconds
	best classifier number 955 with 888 errors and weighed error 0.30335224834872365
	parity 1
	false positives so far: 1369 or 0.6845

	round 6:
4000
	done in 7.13

	done in 6.420959234237671 seconds
	best classifier number 1069 with 1308 errors and weighed error 0.3942172263785378
	parity -1
	false positives so far: 1284 or 0.642

	round 47:
4000
	done in 6.512202978134155 seconds
	best classifier number 29 with 1259 errors and weighed error 0.3893137967136859
	parity 1
	false positives so far: 1315 or 0.6575

	round 48:
4000
	done in 6.817421913146973 seconds
	best classifier number 0 with 1449 errors and weighed error 0.3895490428533239
	parity 1
	false positives so far: 1309 or 0.6545

	round 49:
4000
	done in 6.6372809410095215 seconds
	best classifier number 808 with 2637 errors and weighed error 0.3778535549965072
	parity -1
	false positives so far: 1306 or 0.653

	round 50:
4000
	done in 6.697929859161377 seconds
	best classifier number 1570 with 970 errors and weighed error 0.39574040907816677
	parity 1
	false positives so far: 1307 or 0.6535

	round 51:
4000
	done in 6.585443735122681 seconds
	best classifier number 590 with 1444 errors 

	done in 5.410834789276123 seconds
	best classifier number 1376 with 787 errors and weighed error 0.3508620718450178
	parity 1
	false positives so far: 1025 or 0.73005698005698

	round 11:
3404
	done in 5.536365032196045 seconds
	best classifier number 267 with 994 errors and weighed error 0.3625066354568021
	parity -1
	false positives so far: 1077 or 0.7670940170940171

	round 12:
3404
	done in 5.4848549365997314 seconds
	best classifier number 640 with 2024 errors and weighed error 0.3547309807175117
	parity -1
	false positives so far: 977 or 0.6958689458689459

	round 13:
3404
	done in 5.583754777908325 seconds
	best classifier number 1236 with 852 errors and weighed error 0.36925835785109073
	parity -1
	false positives so far: 936 or 0.6666666666666666

	round 14:
3404
	done in 5.651199102401733 seconds
	best classifier number 1072 with 567 errors and weighed error 0.374416085002934
	parity -1
	false positives so far: 963 or 0.6858974358974359

	round 15:
3404
	done in 5.5511617660

	done in 5.568764925003052 seconds
	best classifier number 1126 with 1242 errors and weighed error 0.38628601471726126
	parity 1
	false positives so far: 1167 or 0.8311965811965812

	round 53:
3404
	done in 5.43697714805603 seconds
	best classifier number 1073 with 1315 errors and weighed error 0.39280646682584325
	parity -1
	false positives so far: 1166 or 0.8304843304843305

	round 54:
3404
	done in 5.4548728466033936 seconds
	best classifier number 984 with 1833 errors and weighed error 0.38009023380610796
	parity 1
	false positives so far: 1143 or 0.8141025641025641

	round 55:
3404
	done in 5.6561360359191895 seconds
	best classifier number 1307 with 1418 errors and weighed error 0.40764100853243873
	parity -1
	false positives so far: 1112 or 0.792022792022792

	round 56:
3404
	done in 5.627604246139526 seconds
	best classifier number 728 with 903 errors and weighed error 0.39663594125196744
	parity -1
	false positives so far: 1130 or 0.8048433048433048

	round 57:
3404
	done in 5

	done in 5.028362989425659 seconds
	best classifier number 1078 with 1894 errors and weighed error 0.35179473703570124
	parity 1
	false positives so far: 942 or 0.8328912466843501

	round 15:
3131
	done in 5.165003061294556 seconds
	best classifier number 1067 with 1636 errors and weighed error 0.3495057661242725
	parity -1
	false positives so far: 994 or 0.878868258178603

	round 16:
3131
	done in 5.223082065582275 seconds
	best classifier number 135 with 1353 errors and weighed error 0.3676296468162115
	parity 1
	false positives so far: 1018 or 0.9000884173297966

	round 17:
3131
	done in 5.025408983230591 seconds
	best classifier number 96 with 563 errors and weighed error 0.3599458724864811
	parity 1
	false positives so far: 1037 or 0.9168877099911583

	round 18:
3131
	done in 5.209611892700195 seconds
	best classifier number 1073 with 1114 errors and weighed error 0.372468597638847
	parity -1
	false positives so far: 1034 or 0.9142351900972591

	round 19:
3131
	done in 5.225486993

	done in 5.273682355880737 seconds
	best classifier number 1073 with 1295 errors and weighed error 0.3977592432168578
	parity -1
	false positives so far: 985 or 0.8709106984969054

	round 57:
3131
	done in 5.264065980911255 seconds
	best classifier number 1102 with 806 errors and weighed error 0.39483591372665694
	parity -1
	false positives so far: 952 or 0.8417329796640142

	round 58:
3131
	done in 5.175098180770874 seconds
	best classifier number 1257 with 1619 errors and weighed error 0.3850828332687182
	parity 1
	false positives so far: 948 or 0.8381962864721485

	round 59:
3131
	done in 5.2053632736206055 seconds
	best classifier number 1551 with 1139 errors and weighed error 0.39559262737973266
	parity 1
	false positives so far: 891 or 0.7877984084880637

	round 60:
3131
	done in 5.2183990478515625 seconds
	best classifier number 1365 with 1614 errors and weighed error 0.38811932934630605
	parity -1
	false positives so far: 893 or 0.7895667550839964

	round 61:
3131
	done in 5.09

	done in 5.080519914627075 seconds
	best classifier number 413 with 1970 errors and weighed error 0.3825489836376044
	parity -1
	false positives so far: 880 or 0.851063829787234

	round 19:
3034
	done in 5.002926826477051 seconds
	best classifier number 1327 with 631 errors and weighed error 0.34510145098106865
	parity -1
	false positives so far: 988 or 0.9555125725338491

	round 20:
3034
	done in 5.067185163497925 seconds
	best classifier number 1245 with 1691 errors and weighed error 0.3684930419235264
	parity 1
	false positives so far: 981 or 0.9487427466150871

	round 21:
3034
	done in 5.077430009841919 seconds
	best classifier number 1042 with 663 errors and weighed error 0.37305409930878297
	parity -1
	false positives so far: 971 or 0.9390715667311412

	round 22:
3034
	done in 4.85624885559082 seconds
	best classifier number 1065 with 2008 errors and weighed error 0.36395096330559246
	parity -1
	false positives so far: 994 or 0.9613152804642167

	round 23:
3034
	done in 4.8685541

	done in 4.945994138717651 seconds
	best classifier number 290 with 1365 errors and weighed error 0.40644211990442086
	parity -1
	false positives so far: 1022 or 0.988394584139265

	round 61:
3034
	done in 5.060459136962891 seconds
	best classifier number 1103 with 570 errors and weighed error 0.40235768831234026
	parity -1
	false positives so far: 1026 or 0.9922630560928434

	round 62:
3034
	done in 5.075117826461792 seconds
	best classifier number 204 with 1364 errors and weighed error 0.3954900913763364
	parity 1
	false positives so far: 1026 or 0.9922630560928434

	round 63:
3034
	done in 5.0715491771698 seconds
	best classifier number 663 with 2019 errors and weighed error 0.4112997884721606
	parity -1
	false positives so far: 1023 or 0.9893617021276596

	round 64:
3034
	done in 5.060585975646973 seconds
	best classifier number 544 with 900 errors and weighed error 0.38734623795161854
	parity -1
	false positives so far: 1019 or 0.9854932301740812

	round 65:
3034
	done in 5.058698

In [523]:
test_face_image_files = glob.glob(os.path.join(DataPath + 'test/FACES', '*.bmp'), recursive=True)

evaluation1 = 0

for i in range(1000):
    # import testing image
    bigimg = loadImg(test_face_image_files[i],IMG_SIZE,IMG_SIZE)

    iimg = integral(bigimg)

    stride = 2 
    increment = 2

    ftrs1 = feature1(iimg, IMG_SIZE, IMG_SIZE, stride, increment)
    ftrs2 = feature2(iimg, IMG_SIZE, IMG_SIZE, stride, increment)
    ftrs3 = feature3(iimg, IMG_SIZE, IMG_SIZE, stride, increment)
    ftrs4 = feature4(iimg, IMG_SIZE, IMG_SIZE, stride, increment)

    fts = ftrs1 + ftrs2 + ftrs3 + ftrs4 

    # evaluate whether there are faces in image
    evaluation1 += evaluate_cascade(fts, fa, fp, ft, fi)

print (evaluation1/1000)

0.999


In [524]:
test_nface_image_files = glob.glob(os.path.join(DataPath + 'test/NFACES', '*.bmp'), recursive=True)

evaluation2 = 0

for i in range(1000):
    # import testing image
    bigimg = loadImg(test_nface_image_files[i],IMG_SIZE,IMG_SIZE)

    iimg = integral(bigimg)

    stride = 2 
    increment = 2

    ftrs1 = feature1(iimg, IMG_SIZE, IMG_SIZE, stride, increment)
    ftrs2 = feature2(iimg, IMG_SIZE, IMG_SIZE, stride, increment)
    ftrs3 = feature3(iimg, IMG_SIZE, IMG_SIZE, stride, increment)
    ftrs4 = feature4(iimg, IMG_SIZE, IMG_SIZE, stride, increment)

    fts = ftrs1 + ftrs2 + ftrs3 + ftrs4 

    # evaluate whether there are faces in image
    evaluation2 += evaluate_cascade(fts, fa, fp, ft, fi)

print (evaluation2/1000)

0.49


In [525]:
featurelists = np.array(features)
(fc, fa, fp, ft, fi) = cascade(5, 80, featurelists, imagesv)

stage 0:
training with 1665 features over 4000 examples
	round 1:
4000
	done in 8.044831037521362 seconds
	best classifier number 1072 with 477 errors and weighed error 0.11925000000000845
	parity -1
	false positives so far: 2000 or 1.0

	round 2:
4000
	done in 7.363533973693848 seconds
	best classifier number 132 with 648 errors and weighed error 0.18803418803419314
	parity 1
	false positives so far: 1659 or 0.8295

	round 3:
4000
	done in 6.992887020111084 seconds
	best classifier number 1129 with 688 errors and weighed error 0.24671417820974234
	parity -1
	false positives so far: 1652 or 0.826

	round 4:
4000
	done in 6.751646995544434 seconds
	best classifier number 1644 with 1129 errors and weighed error 0.28193276508842147
	parity 1
	false positives so far: 1655 or 0.8275

	round 5:
4000
	done in 6.658088207244873 seconds
	best classifier number 955 with 888 errors and weighed error 0.30335224834872365
	parity 1
	false positives so far: 1369 or 0.6845

	round 6:
4000
	done in 6.7

	done in 6.696056842803955 seconds
	best classifier number 1069 with 1308 errors and weighed error 0.3942172263785378
	parity -1
	false positives so far: 1284 or 0.642

	round 47:
4000
	done in 6.623673677444458 seconds
	best classifier number 29 with 1259 errors and weighed error 0.3893137967136859
	parity 1
	false positives so far: 1315 or 0.6575

	round 48:
4000
	done in 6.8058390617370605 seconds
	best classifier number 0 with 1449 errors and weighed error 0.3895490428533239
	parity 1
	false positives so far: 1309 or 0.6545

	round 49:
4000
	done in 6.670332193374634 seconds
	best classifier number 808 with 2637 errors and weighed error 0.3778535549965072
	parity -1
	false positives so far: 1306 or 0.653

	round 50:
4000
	done in 6.885180950164795 seconds
	best classifier number 1570 with 970 errors and weighed error 0.39574040907816677
	parity 1
	false positives so far: 1307 or 0.6535

	round 51:
4000
	done in 6.71050500869751 seconds
	best classifier number 590 with 1444 errors a

	done in 5.703574895858765 seconds
	best classifier number 1376 with 787 errors and weighed error 0.3508620718450178
	parity 1
	false positives so far: 1025 or 0.73005698005698

	round 11:
3404
	done in 5.7867748737335205 seconds
	best classifier number 267 with 994 errors and weighed error 0.3625066354568021
	parity -1
	false positives so far: 1077 or 0.7670940170940171

	round 12:
3404
	done in 5.655127763748169 seconds
	best classifier number 640 with 2024 errors and weighed error 0.3547309807175117
	parity -1
	false positives so far: 977 or 0.6958689458689459

	round 13:
3404
	done in 5.7280049324035645 seconds
	best classifier number 1236 with 852 errors and weighed error 0.36925835785109073
	parity -1
	false positives so far: 936 or 0.6666666666666666

	round 14:
3404
	done in 5.912738084793091 seconds
	best classifier number 1072 with 567 errors and weighed error 0.374416085002934
	parity -1
	false positives so far: 963 or 0.6858974358974359

	round 15:
3404
	done in 5.660939216

	done in 5.847805023193359 seconds
	best classifier number 1126 with 1242 errors and weighed error 0.38628601471726126
	parity 1
	false positives so far: 1167 or 0.8311965811965812

	round 53:
3404
	done in 5.608809947967529 seconds
	best classifier number 1073 with 1315 errors and weighed error 0.39280646682584325
	parity -1
	false positives so far: 1166 or 0.8304843304843305

	round 54:
3404
	done in 5.622588157653809 seconds
	best classifier number 984 with 1833 errors and weighed error 0.38009023380610796
	parity 1
	false positives so far: 1143 or 0.8141025641025641

	round 55:
3404
	done in 5.683219909667969 seconds
	best classifier number 1307 with 1418 errors and weighed error 0.40764100853243873
	parity -1
	false positives so far: 1112 or 0.792022792022792

	round 56:
3404
	done in 5.636402130126953 seconds
	best classifier number 728 with 903 errors and weighed error 0.39663594125196744
	parity -1
	false positives so far: 1130 or 0.8048433048433048

	round 57:
3404
	done in 5.

	done in 5.398818016052246 seconds
	best classifier number 1078 with 1894 errors and weighed error 0.35179473703570124
	parity 1
	false positives so far: 942 or 0.8328912466843501

	round 15:
3131
	done in 5.260264873504639 seconds
	best classifier number 1067 with 1636 errors and weighed error 0.3495057661242725
	parity -1
	false positives so far: 994 or 0.878868258178603

	round 16:
3131
	done in 5.3834922313690186 seconds
	best classifier number 135 with 1353 errors and weighed error 0.3676296468162115
	parity 1
	false positives so far: 1018 or 0.9000884173297966

	round 17:
3131
	done in 5.306465148925781 seconds
	best classifier number 96 with 563 errors and weighed error 0.3599458724864811
	parity 1
	false positives so far: 1037 or 0.9168877099911583

	round 18:
3131
	done in 5.349351167678833 seconds
	best classifier number 1073 with 1114 errors and weighed error 0.372468597638847
	parity -1
	false positives so far: 1034 or 0.9142351900972591

	round 19:
3131
	done in 5.25502324

	done in 5.267512083053589 seconds
	best classifier number 1073 with 1295 errors and weighed error 0.3977592432168578
	parity -1
	false positives so far: 985 or 0.8709106984969054

	round 57:
3131
	done in 5.03196120262146 seconds
	best classifier number 1102 with 806 errors and weighed error 0.39483591372665694
	parity -1
	false positives so far: 952 or 0.8417329796640142

	round 58:
3131
	done in 5.230682134628296 seconds
	best classifier number 1257 with 1619 errors and weighed error 0.3850828332687182
	parity 1
	false positives so far: 948 or 0.8381962864721485

	round 59:
3131
	done in 5.339460849761963 seconds
	best classifier number 1551 with 1139 errors and weighed error 0.39559262737973266
	parity 1
	false positives so far: 891 or 0.7877984084880637

	round 60:
3131
	done in 5.299944877624512 seconds
	best classifier number 1365 with 1614 errors and weighed error 0.38811932934630605
	parity -1
	false positives so far: 893 or 0.7895667550839964

	round 61:
3131
	done in 5.31666

	done in 5.095278024673462 seconds
	best classifier number 413 with 1970 errors and weighed error 0.3825489836376044
	parity -1
	false positives so far: 880 or 0.851063829787234

	round 19:
3034
	done in 5.000233173370361 seconds
	best classifier number 1327 with 631 errors and weighed error 0.34510145098106865
	parity -1
	false positives so far: 988 or 0.9555125725338491

	round 20:
3034
	done in 4.965277910232544 seconds
	best classifier number 1245 with 1691 errors and weighed error 0.3684930419235264
	parity 1
	false positives so far: 981 or 0.9487427466150871

	round 21:
3034
	done in 5.016778945922852 seconds
	best classifier number 1042 with 663 errors and weighed error 0.37305409930878297
	parity -1
	false positives so far: 971 or 0.9390715667311412

	round 22:
3034
	done in 5.035792827606201 seconds
	best classifier number 1065 with 2008 errors and weighed error 0.36395096330559246
	parity -1
	false positives so far: 994 or 0.9613152804642167

	round 23:
3034
	done in 5.071977

	done in 4.740722894668579 seconds
	best classifier number 290 with 1365 errors and weighed error 0.40644211990442086
	parity -1
	false positives so far: 1022 or 0.988394584139265

	round 61:
3034
	done in 4.761940002441406 seconds
	best classifier number 1103 with 570 errors and weighed error 0.40235768831234026
	parity -1
	false positives so far: 1026 or 0.9922630560928434

	round 62:
3034
	done in 4.7102861404418945 seconds
	best classifier number 204 with 1364 errors and weighed error 0.3954900913763364
	parity 1
	false positives so far: 1026 or 0.9922630560928434

	round 63:
3034
	done in 4.757691144943237 seconds
	best classifier number 663 with 2019 errors and weighed error 0.4112997884721606
	parity -1
	false positives so far: 1023 or 0.9893617021276596

	round 64:
3034
	done in 4.733082056045532 seconds
	best classifier number 544 with 900 errors and weighed error 0.38734623795161854
	parity -1
	false positives so far: 1019 or 0.9854932301740812

	round 65:
3034
	done in 60.05

	done in 5.2103431224823 seconds
	best classifier number 1235 with 1547 errors and weighed error 0.3873540396615602
	parity 1
	false positives so far: 981 or 0.9959390862944163

	round 23:
2985
	done in 5.187214136123657 seconds
	best classifier number 1072 with 520 errors and weighed error 0.3734799681356214
	parity -1
	false positives so far: 976 or 0.9908629441624366

	round 24:
2985
	done in 4.953997850418091 seconds
	best classifier number 395 with 1882 errors and weighed error 0.3620939769350222
	parity -1
	false positives so far: 979 or 0.9939086294416244

	round 25:
2985
	done in 5.10607385635376 seconds
	best classifier number 29 with 761 errors and weighed error 0.3572241333337275
	parity 1
	false positives so far: 984 or 0.9989847715736041

	round 26:
2985
	done in 5.134875059127808 seconds
	best classifier number 1562 with 824 errors and weighed error 0.3899177214814683
	parity 1
	false positives so far: 984 or 0.9989847715736041

	round 27:
2985
	done in 5.11560320854187 s

	done in 5.0891358852386475 seconds
	best classifier number 589 with 1952 errors and weighed error 0.3904026319114416
	parity 1
	false positives so far: 937 or 0.9512690355329949

	round 66:
2985
	done in 5.0574610233306885 seconds
	best classifier number 1509 with 859 errors and weighed error 0.4024558683129483
	parity 1
	false positives so far: 939 or 0.9532994923857868

	round 67:
2985
	done in 5.1622231006622314 seconds
	best classifier number 1223 with 1271 errors and weighed error 0.3950584138100939
	parity 1
	false positives so far: 939 or 0.9532994923857868

	round 68:
2985
	done in 5.0595009326934814 seconds
	best classifier number 1073 with 1364 errors and weighed error 0.4044424927944402
	parity -1
	false positives so far: 938 or 0.9522842639593908

	round 69:
2985
	done in 5.127405166625977 seconds
	best classifier number 1658 with 983 errors and weighed error 0.37952544719943404
	parity -1
	false positives so far: 969 or 0.983756345177665

	round 70:
2985
	done in 4.933825

In [526]:
test_face_image_files = glob.glob(os.path.join(DataPath + 'test/FACES', '*.bmp'), recursive=True)

evaluation1 = 0

for i in range(1000):
    # import testing image
    bigimg = loadImg(test_face_image_files[i],IMG_SIZE,IMG_SIZE)

    iimg = integral(bigimg)

    stride = 2 
    increment = 2

    ftrs1 = feature1(iimg, IMG_SIZE, IMG_SIZE, stride, increment)
    ftrs2 = feature2(iimg, IMG_SIZE, IMG_SIZE, stride, increment)
    ftrs3 = feature3(iimg, IMG_SIZE, IMG_SIZE, stride, increment)
    ftrs4 = feature4(iimg, IMG_SIZE, IMG_SIZE, stride, increment)

    fts = ftrs1 + ftrs2 + ftrs3 + ftrs4 

    # evaluate whether there are faces in image
    evaluation1 += evaluate_cascade(fts, fa, fp, ft, fi)

print (evaluation1/1000)

0.999


In [527]:
test_nface_image_files = glob.glob(os.path.join(DataPath + 'test/NFACES', '*.bmp'), recursive=True)

evaluation2 = 0

for i in range(1000):
    # import testing image
    bigimg = loadImg(test_nface_image_files[i],IMG_SIZE,IMG_SIZE)

    iimg = integral(bigimg)

    stride = 2 
    increment = 2

    ftrs1 = feature1(iimg, IMG_SIZE, IMG_SIZE, stride, increment)
    ftrs2 = feature2(iimg, IMG_SIZE, IMG_SIZE, stride, increment)
    ftrs3 = feature3(iimg, IMG_SIZE, IMG_SIZE, stride, increment)
    ftrs4 = feature4(iimg, IMG_SIZE, IMG_SIZE, stride, increment)

    fts = ftrs1 + ftrs2 + ftrs3 + ftrs4 

    # evaluate whether there are faces in image
    evaluation2 += evaluate_cascade(fts, fa, fp, ft, fi)

print (evaluation2/1000)

0.47


In [528]:
featurelists = np.array(features)
(fc, fa, fp, ft, fi) = cascade(6, 80, featurelists, imagesv)

stage 0:
training with 1665 features over 4000 examples
	round 1:
4000
	done in 7.855165958404541 seconds
	best classifier number 1072 with 477 errors and weighed error 0.11925000000000845
	parity -1
	false positives so far: 2000 or 1.0

	round 2:
4000
	done in 7.268079042434692 seconds
	best classifier number 132 with 648 errors and weighed error 0.18803418803419314
	parity 1
	false positives so far: 1659 or 0.8295

	round 3:
4000
	done in 7.031923055648804 seconds
	best classifier number 1129 with 688 errors and weighed error 0.24671417820974234
	parity -1
	false positives so far: 1652 or 0.826

	round 4:
4000
	done in 6.96855902671814 seconds
	best classifier number 1644 with 1129 errors and weighed error 0.28193276508842147
	parity 1
	false positives so far: 1655 or 0.8275

	round 5:
4000
	done in 7.127329111099243 seconds
	best classifier number 955 with 888 errors and weighed error 0.30335224834872365
	parity 1
	false positives so far: 1369 or 0.6845

	round 6:
4000
	done in 7.19

	done in 6.848540782928467 seconds
	best classifier number 1069 with 1308 errors and weighed error 0.3942172263785378
	parity -1
	false positives so far: 1284 or 0.642

	round 47:
4000
	done in 6.977797031402588 seconds
	best classifier number 29 with 1259 errors and weighed error 0.3893137967136859
	parity 1
	false positives so far: 1315 or 0.6575

	round 48:
4000
	done in 7.19135594367981 seconds
	best classifier number 0 with 1449 errors and weighed error 0.3895490428533239
	parity 1
	false positives so far: 1309 or 0.6545

	round 49:
4000
	done in 6.926468849182129 seconds
	best classifier number 808 with 2637 errors and weighed error 0.3778535549965072
	parity -1
	false positives so far: 1306 or 0.653

	round 50:
4000
	done in 7.083591938018799 seconds
	best classifier number 1570 with 970 errors and weighed error 0.39574040907816677
	parity 1
	false positives so far: 1307 or 0.6535

	round 51:
4000
	done in 6.948729038238525 seconds
	best classifier number 590 with 1444 errors an

	done in 5.3536272048950195 seconds
	best classifier number 1376 with 787 errors and weighed error 0.3508620718450178
	parity 1
	false positives so far: 1025 or 0.73005698005698

	round 11:
3404
	done in 5.426868915557861 seconds
	best classifier number 267 with 994 errors and weighed error 0.3625066354568021
	parity -1
	false positives so far: 1077 or 0.7670940170940171

	round 12:
3404
	done in 5.351020097732544 seconds
	best classifier number 640 with 2024 errors and weighed error 0.3547309807175117
	parity -1
	false positives so far: 977 or 0.6958689458689459

	round 13:
3404
	done in 5.446092844009399 seconds
	best classifier number 1236 with 852 errors and weighed error 0.36925835785109073
	parity -1
	false positives so far: 936 or 0.6666666666666666

	round 14:
3404
	done in 5.485108137130737 seconds
	best classifier number 1072 with 567 errors and weighed error 0.374416085002934
	parity -1
	false positives so far: 963 or 0.6858974358974359

	round 15:
3404
	done in 5.4886701107

	done in 5.427334785461426 seconds
	best classifier number 1126 with 1242 errors and weighed error 0.38628601471726126
	parity 1
	false positives so far: 1167 or 0.8311965811965812

	round 53:
3404
	done in 5.4712488651275635 seconds
	best classifier number 1073 with 1315 errors and weighed error 0.39280646682584325
	parity -1
	false positives so far: 1166 or 0.8304843304843305

	round 54:
3404
	done in 5.5025599002838135 seconds
	best classifier number 984 with 1833 errors and weighed error 0.38009023380610796
	parity 1
	false positives so far: 1143 or 0.8141025641025641

	round 55:
3404
	done in 5.521533012390137 seconds
	best classifier number 1307 with 1418 errors and weighed error 0.40764100853243873
	parity -1
	false positives so far: 1112 or 0.792022792022792

	round 56:
3404
	done in 5.525507211685181 seconds
	best classifier number 728 with 903 errors and weighed error 0.39663594125196744
	parity -1
	false positives so far: 1130 or 0.8048433048433048

	round 57:
3404
	done in 

	done in 5.007225275039673 seconds
	best classifier number 1078 with 1894 errors and weighed error 0.35179473703570124
	parity 1
	false positives so far: 942 or 0.8328912466843501

	round 15:
3131
	done in 5.2606141567230225 seconds
	best classifier number 1067 with 1636 errors and weighed error 0.3495057661242725
	parity -1
	false positives so far: 994 or 0.878868258178603

	round 16:
3131
	done in 5.399254083633423 seconds
	best classifier number 135 with 1353 errors and weighed error 0.3676296468162115
	parity 1
	false positives so far: 1018 or 0.9000884173297966

	round 17:
3131
	done in 5.1451311111450195 seconds
	best classifier number 96 with 563 errors and weighed error 0.3599458724864811
	parity 1
	false positives so far: 1037 or 0.9168877099911583

	round 18:
3131
	done in 5.317065954208374 seconds
	best classifier number 1073 with 1114 errors and weighed error 0.372468597638847
	parity -1
	false positives so far: 1034 or 0.9142351900972591

	round 19:
3131
	done in 5.3119249

...

	done in 5.1733739376068115 seconds
	best classifier number 290 with 1365 errors and weighed error 0.40644211990442086
	parity -1
	false positives so far: 1022 or 0.988394584139265

	round 61:
3034
	done in 5.142401933670044 seconds
	best classifier number 1103 with 570 errors and weighed error 0.40235768831234026
	parity -1
	false positives so far: 1026 or 0.9922630560928434

	round 62:
3034
	done in 5.081245183944702 seconds
	best classifier number 204 with 1364 errors and weighed error 0.3954900913763364
	parity 1
	false positives so far: 1026 or 0.9922630560928434

	round 63:
3034
	done in 4.973718166351318 seconds
	best classifier number 663 with 2019 errors and weighed error 0.4112997884721606
	parity -1
	false positives so far: 1023 or 0.9893617021276596

	round 64:
3034
	done in 5.027716875076294 seconds
	best classifier number 544 with 900 errors and weighed error 0.38734623795161854
	parity -1
	false positives so far: 1019 or 0.9854932301740812

	round 65:
3034
	done in 5.088

	done in 5.103612899780273 seconds
	best classifier number 1235 with 1547 errors and weighed error 0.3873540396615602
	parity 1
	false positives so far: 981 or 0.9959390862944163

	round 23:
2985
	done in 5.01629900932312 seconds
	best classifier number 1072 with 520 errors and weighed error 0.3734799681356214
	parity -1
	false positives so far: 976 or 0.9908629441624366

	round 24:
2985
	done in 4.889452695846558 seconds
	best classifier number 395 with 1882 errors and weighed error 0.3620939769350222
	parity -1
	false positives so far: 979 or 0.9939086294416244

	round 25:
2985
	done in 5.136836051940918 seconds
	best classifier number 29 with 761 errors and weighed error 0.3572241333337275
	parity 1
	false positives so far: 984 or 0.9989847715736041

	round 26:
2985
	done in 5.128347158432007 seconds
	best classifier number 1562 with 824 errors and weighed error 0.3899177214814683
	parity 1
	false positives so far: 984 or 0.9989847715736041

	round 27:
2985
	done in 5.08859014511108

	done in 5.2249720096588135 seconds
	best classifier number 638 with 652 errors and weighed error 0.3904206207258051
	parity -1
	false positives so far: 934 or 0.9482233502538071

	round 65:
2985
	done in 5.250621795654297 seconds
	best classifier number 589 with 1952 errors and weighed error 0.3904026319114416
	parity 1
	false positives so far: 937 or 0.9512690355329949

	round 66:
2985
	done in 5.0787787437438965 seconds
	best classifier number 1509 with 859 errors and weighed error 0.4024558683129483
	parity 1
	false positives so far: 939 or 0.9532994923857868

	round 67:
2985
	done in 5.217534780502319 seconds
	best classifier number 1223 with 1271 errors and weighed error 0.3950584138100939
	parity 1
	false positives so far: 939 or 0.9532994923857868

	round 68:
2985
	done in 5.08732795715332 seconds
	best classifier number 1073 with 1364 errors and weighed error 0.4044424927944402
	parity -1
	false positives so far: 938 or 0.9522842639593908

	round 69:
2985
	done in 4.7593600749

	done in 4.727715253829956 seconds
	best classifier number 1366 with 627 errors and weighed error 0.3804843697808598
	parity -1
	false positives so far: 979 or 0.9949186991869918

	round 27:
2984
	done in 4.726827144622803 seconds
	best classifier number 1034 with 2026 errors and weighed error 0.37516285087736384
	parity -1
	false positives so far: 978 or 0.9939024390243902

	round 28:
2984
	done in 4.7305707931518555 seconds
	best classifier number 955 with 720 errors and weighed error 0.3712935811201808
	parity 1
	false positives so far: 971 or 0.9867886178861789

	round 29:
2984
	done in 4.717194080352783 seconds
	best classifier number 839 with 976 errors and weighed error 0.3779795088143635
	parity 1
	false positives so far: 975 or 0.9908536585365854

	round 30:
2984
	done in 4.77580189704895 seconds
	best classifier number 1073 with 1107 errors and weighed error 0.39038663226693315
	parity -1
	false positives so far: 979 or 0.9949186991869918

	round 31:
2984
	done in 4.764446973

	done in 4.728883743286133 seconds
	best classifier number 1323 with 1886 errors and weighed error 0.40199198978077555
	parity 1
	false positives so far: 983 or 0.9989837398373984

	round 70:
2984
	done in 4.702963829040527 seconds
	best classifier number 1654 with 1084 errors and weighed error 0.3865698923055724
	parity -1
	false positives so far: 978 or 0.9939024390243902

	round 71:
2984
	done in 4.743995904922485 seconds
	best classifier number 1562 with 867 errors and weighed error 0.39809011475787554
	parity 1
	false positives so far: 979 or 0.9949186991869918

	round 72:
2984
	done in 4.7315850257873535 seconds
	best classifier number 1551 with 1052 errors and weighed error 0.39817933748604334
	parity 1
	false positives so far: 980 or 0.9959349593495935

	round 73:
2984
	done in 4.718356132507324 seconds
	best classifier number 1388 with 1987 errors and weighed error 0.3854699335789584
	parity -1
	false positives so far: 981 or 0.9969512195121951

	round 74:
2984
	done in 4.9518

In [529]:
test_face_image_files = glob.glob(os.path.join(DataPath + 'test/FACES', '*.bmp'), recursive=True)

evaluation1 = 0

for i in range(1000):
    # import testing image
    bigimg = loadImg(test_face_image_files[i],IMG_SIZE,IMG_SIZE)

    iimg = integral(bigimg)

    stride = 2 
    increment = 2

    ftrs1 = feature1(iimg, IMG_SIZE, IMG_SIZE, stride, increment)
    ftrs2 = feature2(iimg, IMG_SIZE, IMG_SIZE, stride, increment)
    ftrs3 = feature3(iimg, IMG_SIZE, IMG_SIZE, stride, increment)
    ftrs4 = feature4(iimg, IMG_SIZE, IMG_SIZE, stride, increment)

    fts = ftrs1 + ftrs2 + ftrs3 + ftrs4 

    # evaluate whether there are faces in image
    evaluation1 += evaluate_cascade(fts, fa, fp, ft, fi)

print (evaluation1/1000)

0.999


In [530]:
test_nface_image_files = glob.glob(os.path.join(DataPath + 'test/NFACES', '*.bmp'), recursive=True)

evaluation2 = 0

for i in range(1000):
    # import testing image
    bigimg = loadImg(test_nface_image_files[i],IMG_SIZE,IMG_SIZE)

    iimg = integral(bigimg)

    stride = 2 
    increment = 2

    ftrs1 = feature1(iimg, IMG_SIZE, IMG_SIZE, stride, increment)
    ftrs2 = feature2(iimg, IMG_SIZE, IMG_SIZE, stride, increment)
    ftrs3 = feature3(iimg, IMG_SIZE, IMG_SIZE, stride, increment)
    ftrs4 = feature4(iimg, IMG_SIZE, IMG_SIZE, stride, increment)

    fts = ftrs1 + ftrs2 + ftrs3 + ftrs4 

    # evaluate whether there are faces in image
    evaluation2 += evaluate_cascade(fts, fa, fp, ft, fi)

print (evaluation2/1000)

0.466


In [531]:
featurelists = np.array(features)
(fc, fa, fp, ft, fi) = cascade(7, 80, featurelists, imagesv)

stage 0:
training with 1665 features over 4000 examples
	round 1:
4000
	done in 7.3540940284729 seconds
	best classifier number 1072 with 477 errors and weighed error 0.11925000000000845
	parity -1
	false positives so far: 2000 or 1.0

	round 2:
4000
	done in 6.952683210372925 seconds
	best classifier number 132 with 648 errors and weighed error 0.18803418803419314
	parity 1
	false positives so far: 1659 or 0.8295

	round 3:
4000
	done in 6.833471059799194 seconds
	best classifier number 1129 with 688 errors and weighed error 0.24671417820974234
	parity -1
	false positives so far: 1652 or 0.826

	round 4:
4000
	done in 6.648910045623779 seconds
	best classifier number 1644 with 1129 errors and weighed error 0.28193276508842147
	parity 1
	false positives so far: 1655 or 0.8275

	round 5:
4000
	done in 6.469988107681274 seconds
	best classifier number 955 with 888 errors and weighed error 0.30335224834872365
	parity 1
	false positives so far: 1369 or 0.6845

	round 6:
4000
	done in 6.519

	done in 6.440423965454102 seconds
	best classifier number 1069 with 1308 errors and weighed error 0.3942172263785378
	parity -1
	false positives so far: 1284 or 0.642

	round 47:
4000
	done in 6.585543870925903 seconds
	best classifier number 29 with 1259 errors and weighed error 0.3893137967136859
	parity 1
	false positives so far: 1315 or 0.6575

	round 48:
4000
	done in 6.690985918045044 seconds
	best classifier number 0 with 1449 errors and weighed error 0.3895490428533239
	parity 1
	false positives so far: 1309 or 0.6545

	round 49:
4000
	done in 6.431098222732544 seconds
	best classifier number 808 with 2637 errors and weighed error 0.3778535549965072
	parity -1
	false positives so far: 1306 or 0.653

	round 50:
4000
	done in 6.531851053237915 seconds
	best classifier number 1570 with 970 errors and weighed error 0.39574040907816677
	parity 1
	false positives so far: 1307 or 0.6535

	round 51:
4000
	done in 6.530577898025513 seconds
	best classifier number 590 with 1444 errors a

	done in 5.776136875152588 seconds
	best classifier number 1376 with 787 errors and weighed error 0.3508620718450178
	parity 1
	false positives so far: 1025 or 0.73005698005698

	round 11:
3404
	done in 5.782170057296753 seconds
	best classifier number 267 with 994 errors and weighed error 0.3625066354568021
	parity -1
	false positives so far: 1077 or 0.7670940170940171

	round 12:
3404
	done in 5.68992805480957 seconds
	best classifier number 640 with 2024 errors and weighed error 0.3547309807175117
	parity -1
	false positives so far: 977 or 0.6958689458689459

	round 13:
3404
	done in 5.7241530418396 seconds
	best classifier number 1236 with 852 errors and weighed error 0.36925835785109073
	parity -1
	false positives so far: 936 or 0.6666666666666666

	round 14:
3404
	done in 5.9111011028289795 seconds
	best classifier number 1072 with 567 errors and weighed error 0.374416085002934
	parity -1
	false positives so far: 963 or 0.6858974358974359

	round 15:
3404
	done in 5.6310951709747

	done in 5.662129878997803 seconds
	best classifier number 1126 with 1242 errors and weighed error 0.38628601471726126
	parity 1
	false positives so far: 1167 or 0.8311965811965812

	round 53:
3404
	done in 5.749207973480225 seconds
	best classifier number 1073 with 1315 errors and weighed error 0.39280646682584325
	parity -1
	false positives so far: 1166 or 0.8304843304843305

	round 54:
3404
	done in 5.582794904708862 seconds
	best classifier number 984 with 1833 errors and weighed error 0.38009023380610796
	parity 1
	false positives so far: 1143 or 0.8141025641025641

	round 55:
3404
	done in 5.4506919384002686 seconds
	best classifier number 1307 with 1418 errors and weighed error 0.40764100853243873
	parity -1
	false positives so far: 1112 or 0.792022792022792

	round 56:
3404
	done in 5.413857698440552 seconds
	best classifier number 728 with 903 errors and weighed error 0.39663594125196744
	parity -1
	false positives so far: 1130 or 0.8048433048433048

	round 57:
3404
	done in 5

	done in 5.264035224914551 seconds
	best classifier number 1078 with 1894 errors and weighed error 0.35179473703570124
	parity 1
	false positives so far: 942 or 0.8328912466843501

	round 15:
3131
	done in 5.137315034866333 seconds
	best classifier number 1067 with 1636 errors and weighed error 0.3495057661242725
	parity -1
	false positives so far: 994 or 0.878868258178603

	round 16:
3131
	done in 5.38715124130249 seconds
	best classifier number 135 with 1353 errors and weighed error 0.3676296468162115
	parity 1
	false positives so far: 1018 or 0.9000884173297966

	round 17:
3131
	done in 5.325000047683716 seconds
	best classifier number 96 with 563 errors and weighed error 0.3599458724864811
	parity 1
	false positives so far: 1037 or 0.9168877099911583

	round 18:
3131
	done in 5.333291053771973 seconds
	best classifier number 1073 with 1114 errors and weighed error 0.372468597638847
	parity -1
	false positives so far: 1034 or 0.9142351900972591

	round 19:
3131
	done in 5.3031480312

...

	done in 5.112783908843994 seconds
	best classifier number 589 with 1952 errors and weighed error 0.3904026319114416
	parity 1
	false positives so far: 937 or 0.9512690355329949

	round 66:
2985
	done in 5.060443162918091 seconds
	best classifier number 1509 with 859 errors and weighed error 0.4024558683129483
	parity 1
	false positives so far: 939 or 0.9532994923857868

	round 67:
2985
	done in 5.043731212615967 seconds
	best classifier number 1223 with 1271 errors and weighed error 0.3950584138100939
	parity 1
	false positives so far: 939 or 0.9532994923857868

	round 68:
2985
	done in 4.957985162734985 seconds
	best classifier number 1073 with 1364 errors and weighed error 0.4044424927944402
	parity -1
	false positives so far: 938 or 0.9522842639593908

	round 69:
2985
	done in 5.015156269073486 seconds
	best classifier number 1658 with 983 errors and weighed error 0.37952544719943404
	parity -1
	false positives so far: 969 or 0.983756345177665

	round 70:
2985
	done in 5.0104889869

	done in 5.162324905395508 seconds
	best classifier number 1034 with 2026 errors and weighed error 0.37516285087736384
	parity -1
	false positives so far: 978 or 0.9939024390243902

	round 28:
2984
	done in 5.339254856109619 seconds
	best classifier number 955 with 720 errors and weighed error 0.3712935811201808
	parity 1
	false positives so far: 971 or 0.9867886178861789

	round 29:
2984
	done in 5.045212745666504 seconds
	best classifier number 839 with 976 errors and weighed error 0.3779795088143635
	parity 1
	false positives so far: 975 or 0.9908536585365854

	round 30:
2984
	done in 5.250939846038818 seconds
	best classifier number 1073 with 1107 errors and weighed error 0.39038663226693315
	parity -1
	false positives so far: 979 or 0.9949186991869918

	round 31:
2984
	done in 4.923863887786865 seconds
	best classifier number 1467 with 877 errors and weighed error 0.36796617365809453
	parity -1
	false positives so far: 981 or 0.9969512195121951

	round 32:
2984
	done in 5.09422492

	done in 4.942823171615601 seconds
	best classifier number 1654 with 1084 errors and weighed error 0.3865698923055724
	parity -1
	false positives so far: 978 or 0.9939024390243902

	round 71:
2984
	done in 4.957771062850952 seconds
	best classifier number 1562 with 867 errors and weighed error 0.39809011475787554
	parity 1
	false positives so far: 979 or 0.9949186991869918

	round 72:
2984
	done in 4.965544939041138 seconds
	best classifier number 1551 with 1052 errors and weighed error 0.39817933748604334
	parity 1
	false positives so far: 980 or 0.9959349593495935

	round 73:
2984
	done in 4.9324140548706055 seconds
	best classifier number 1388 with 1987 errors and weighed error 0.3854699335789584
	parity -1
	false positives so far: 981 or 0.9969512195121951

	round 74:
2984
	done in 4.937203407287598 seconds
	best classifier number 318 with 884 errors and weighed error 0.3941851246360907
	parity 1
	false positives so far: 967 or 0.9827235772357723

	round 75:
2984
	done in 4.9111511

	done in 5.032311916351318 seconds
	best classifier number 1245 with 1278 errors and weighed error 0.3698610990466108
	parity 1
	false positives so far: 975 or 0.9989754098360656

	round 33:
2976
	done in 5.099675893783569 seconds
	best classifier number 1376 with 798 errors and weighed error 0.38261612351968716
	parity 1
	false positives so far: 975 or 0.9989754098360656

	round 34:
2976
	done in 5.084450006484985 seconds
	best classifier number 281 with 975 errors and weighed error 0.3837175943423802
	parity -1
	false positives so far: 975 or 0.9989754098360656

	round 35:
2976
	done in 4.909726142883301 seconds
	best classifier number 1103 with 563 errors and weighed error 0.38958577733949346
	parity -1
	false positives so far: 975 or 0.9989754098360656

	round 36:
2976
	done in 4.924911975860596 seconds
	best classifier number 1361 with 1669 errors and weighed error 0.39059241898456737
	parity 1
	false positives so far: 975 or 0.9989754098360656

	round 37:
2976
	done in 4.82878708

	done in 4.885048151016235 seconds
	best classifier number 0 with 851 errors and weighed error 0.40817609010662964
	parity 1
	false positives so far: 940 or 0.9631147540983607

	round 75:
2976
	done in 4.832188367843628 seconds
	best classifier number 182 with 1394 errors and weighed error 0.405356966513906
	parity -1
	false positives so far: 940 or 0.9631147540983607

	round 76:
2976
	done in 4.867840051651001 seconds
	best classifier number 79 with 701 errors and weighed error 0.4013175973810257
	parity 1
	false positives so far: 946 or 0.9692622950819673

	round 77:
2976
	done in 4.862406253814697 seconds
	best classifier number 1615 with 1049 errors and weighed error 0.3888822760832363
	parity 1
	false positives so far: 943 or 0.9661885245901639

	round 78:
2976
	done in 4.832459926605225 seconds
	best classifier number 102 with 1611 errors and weighed error 0.4103053143785421
	parity 1
	false positives so far: 949 or 0.9723360655737705

	round 79:
2976
	done in 4.844088792800903 s

In [532]:
test_face_image_files = glob.glob(os.path.join(DataPath + 'test/FACES', '*.bmp'), recursive=True)

evaluation1 = 0

for i in range(1000):
    # import testing image
    bigimg = loadImg(test_face_image_files[i],IMG_SIZE,IMG_SIZE)

    iimg = integral(bigimg)

    stride = 2 
    increment = 2

    ftrs1 = feature1(iimg, IMG_SIZE, IMG_SIZE, stride, increment)
    ftrs2 = feature2(iimg, IMG_SIZE, IMG_SIZE, stride, increment)
    ftrs3 = feature3(iimg, IMG_SIZE, IMG_SIZE, stride, increment)
    ftrs4 = feature4(iimg, IMG_SIZE, IMG_SIZE, stride, increment)

    fts = ftrs1 + ftrs2 + ftrs3 + ftrs4 

    # evaluate whether there are faces in image
    evaluation1 += evaluate_cascade(fts, fa, fp, ft, fi)

print (evaluation1/1000)

0.999


In [533]:
test_nface_image_files = glob.glob(os.path.join(DataPath + 'test/NFACES', '*.bmp'), recursive=True)

evaluation2 = 0

for i in range(1000):
    # import testing image
    bigimg = loadImg(test_nface_image_files[i],IMG_SIZE,IMG_SIZE)

    iimg = integral(bigimg)

    stride = 2 
    increment = 2

    ftrs1 = feature1(iimg, IMG_SIZE, IMG_SIZE, stride, increment)
    ftrs2 = feature2(iimg, IMG_SIZE, IMG_SIZE, stride, increment)
    ftrs3 = feature3(iimg, IMG_SIZE, IMG_SIZE, stride, increment)
    ftrs4 = feature4(iimg, IMG_SIZE, IMG_SIZE, stride, increment)

    fts = ftrs1 + ftrs2 + ftrs3 + ftrs4 

    # evaluate whether there are faces in image
    evaluation2 += evaluate_cascade(fts, fa, fp, ft, fi)

print (evaluation2/1000)

0.45


In [534]:
featurelists = np.array(features)
(fc, fa, fp, ft, fi) = cascade(8, 80, featurelists, imagesv)

stage 0:
training with 1665 features over 4000 examples
	round 1:
4000
	done in 7.697959899902344 seconds
	best classifier number 1072 with 477 errors and weighed error 0.11925000000000845
	parity -1
	false positives so far: 2000 or 1.0

	round 2:
4000
	done in 7.053429841995239 seconds
	best classifier number 132 with 648 errors and weighed error 0.18803418803419314
	parity 1
	false positives so far: 1659 or 0.8295

	round 3:
4000
	done in 7.17329216003418 seconds
	best classifier number 1129 with 688 errors and weighed error 0.24671417820974234
	parity -1
	false positives so far: 1652 or 0.826

	round 4:
4000
	done in 7.081411123275757 seconds
	best classifier number 1644 with 1129 errors and weighed error 0.28193276508842147
	parity 1
	false positives so far: 1655 or 0.8275

	round 5:
4000
	done in 6.9699482917785645 seconds
	best classifier number 955 with 888 errors and weighed error 0.30335224834872365
	parity 1
	false positives so far: 1369 or 0.6845

	round 6:
4000
	done in 7.2

	done in 6.693018913269043 seconds
	best classifier number 1069 with 1308 errors and weighed error 0.3942172263785378
	parity -1
	false positives so far: 1284 or 0.642

	round 47:
4000
	done in 6.881356954574585 seconds
	best classifier number 29 with 1259 errors and weighed error 0.3893137967136859
	parity 1
	false positives so far: 1315 or 0.6575

	round 48:
4000
	done in 7.055878162384033 seconds
	best classifier number 0 with 1449 errors and weighed error 0.3895490428533239
	parity 1
	false positives so far: 1309 or 0.6545

	round 49:
4000
	done in 6.970310211181641 seconds
	best classifier number 808 with 2637 errors and weighed error 0.3778535549965072
	parity -1
	false positives so far: 1306 or 0.653

	round 50:
4000
	done in 6.855123996734619 seconds
	best classifier number 1570 with 970 errors and weighed error 0.39574040907816677
	parity 1
	false positives so far: 1307 or 0.6535

	round 51:
4000
	done in 6.871351003646851 seconds
	best classifier number 590 with 1444 errors a

	done in 5.60212516784668 seconds
	best classifier number 1376 with 787 errors and weighed error 0.3508620718450178
	parity 1
	false positives so far: 1025 or 0.73005698005698

	round 11:
3404
	done in 5.651720285415649 seconds
	best classifier number 267 with 994 errors and weighed error 0.3625066354568021
	parity -1
	false positives so far: 1077 or 0.7670940170940171

	round 12:
3404
	done in 5.647625207901001 seconds
	best classifier number 640 with 2024 errors and weighed error 0.3547309807175117
	parity -1
	false positives so far: 977 or 0.6958689458689459

	round 13:
3404
	done in 5.772561073303223 seconds
	best classifier number 1236 with 852 errors and weighed error 0.36925835785109073
	parity -1
	false positives so far: 936 or 0.6666666666666666

	round 14:
3404
	done in 5.647264003753662 seconds
	best classifier number 1072 with 567 errors and weighed error 0.374416085002934
	parity -1
	false positives so far: 963 or 0.6858974358974359

	round 15:
3404
	done in 5.588985919952

	done in 5.574041843414307 seconds
	best classifier number 1126 with 1242 errors and weighed error 0.38628601471726126
	parity 1
	false positives so far: 1167 or 0.8311965811965812

	round 53:
3404
	done in 5.720081806182861 seconds
	best classifier number 1073 with 1315 errors and weighed error 0.39280646682584325
	parity -1
	false positives so far: 1166 or 0.8304843304843305

	round 54:
3404
	done in 5.764231204986572 seconds
	best classifier number 984 with 1833 errors and weighed error 0.38009023380610796
	parity 1
	false positives so far: 1143 or 0.8141025641025641

	round 55:
3404
	done in 5.889388084411621 seconds
	best classifier number 1307 with 1418 errors and weighed error 0.40764100853243873
	parity -1
	false positives so far: 1112 or 0.792022792022792

	round 56:
3404
	done in 5.7230589389801025 seconds
	best classifier number 728 with 903 errors and weighed error 0.39663594125196744
	parity -1
	false positives so far: 1130 or 0.8048433048433048

	round 57:
3404
	done in 5

	done in 5.201985836029053 seconds
	best classifier number 1078 with 1894 errors and weighed error 0.35179473703570124
	parity 1
	false positives so far: 942 or 0.8328912466843501

	round 15:
3131
	done in 5.439574956893921 seconds
	best classifier number 1067 with 1636 errors and weighed error 0.3495057661242725
	parity -1
	false positives so far: 994 or 0.878868258178603

	round 16:
3131
	done in 5.3767759799957275 seconds
	best classifier number 135 with 1353 errors and weighed error 0.3676296468162115
	parity 1
	false positives so far: 1018 or 0.9000884173297966

	round 17:
3131
	done in 5.211088180541992 seconds
	best classifier number 96 with 563 errors and weighed error 0.3599458724864811
	parity 1
	false positives so far: 1037 or 0.9168877099911583

	round 18:
3131
	done in 5.43720006942749 seconds
	best classifier number 1073 with 1114 errors and weighed error 0.372468597638847
	parity -1
	false positives so far: 1034 or 0.9142351900972591

	round 19:
3131
	done in 5.342031717

...

	done in 4.917418003082275 seconds
	best classifier number 1323 with 1886 errors and weighed error 0.40199198978077555
	parity 1
	false positives so far: 983 or 0.9989837398373984

	round 70:
2984
	done in 4.904268026351929 seconds
	best classifier number 1654 with 1084 errors and weighed error 0.3865698923055724
	parity -1
	false positives so far: 978 or 0.9939024390243902

	round 71:
2984
	done in 5.066864967346191 seconds
	best classifier number 1562 with 867 errors and weighed error 0.39809011475787554
	parity 1
	false positives so far: 979 or 0.9949186991869918

	round 72:
2984
	done in 4.991617202758789 seconds
	best classifier number 1551 with 1052 errors and weighed error 0.39817933748604334
	parity 1
	false positives so far: 980 or 0.9959349593495935

	round 73:
2984
	done in 5.072983980178833 seconds
	best classifier number 1388 with 1987 errors and weighed error 0.3854699335789584
	parity -1
	false positives so far: 981 or 0.9969512195121951

	round 74:
2984
	done in 5.04544

	done in 4.977158069610596 seconds
	best classifier number 212 with 964 errors and weighed error 0.373716730857231
	parity 1
	false positives so far: 975 or 0.9989754098360656

	round 32:
2976
	done in 4.970026969909668 seconds
	best classifier number 1245 with 1278 errors and weighed error 0.3698610990466108
	parity 1
	false positives so far: 975 or 0.9989754098360656

	round 33:
2976
	done in 4.870265960693359 seconds
	best classifier number 1376 with 798 errors and weighed error 0.38261612351968716
	parity 1
	false positives so far: 975 or 0.9989754098360656

	round 34:
2976
	done in 4.994250059127808 seconds
	best classifier number 281 with 975 errors and weighed error 0.3837175943423802
	parity -1
	false positives so far: 975 or 0.9989754098360656

	round 35:
2976
	done in 5.042987108230591 seconds
	best classifier number 1103 with 563 errors and weighed error 0.38958577733949346
	parity -1
	false positives so far: 975 or 0.9989754098360656

	round 36:
2976
	done in 5.017734289169

	done in 4.919277906417847 seconds
	best classifier number 1035 with 1413 errors and weighed error 0.3997141570307392
	parity -1
	false positives so far: 915 or 0.9375

	round 74:
2976
	done in 5.084506034851074 seconds
	best classifier number 0 with 851 errors and weighed error 0.40817609010662964
	parity 1
	false positives so far: 940 or 0.9631147540983607

	round 75:
2976
	done in 5.125343322753906 seconds
	best classifier number 182 with 1394 errors and weighed error 0.405356966513906
	parity -1
	false positives so far: 940 or 0.9631147540983607

	round 76:
2976
	done in 5.057080030441284 seconds
	best classifier number 79 with 701 errors and weighed error 0.4013175973810257
	parity 1
	false positives so far: 946 or 0.9692622950819673

	round 77:
2976
	done in 4.781372785568237 seconds
	best classifier number 1615 with 1049 errors and weighed error 0.3888822760832363
	parity 1
	false positives so far: 943 or 0.9661885245901639

	round 78:
2976
	done in 4.759865045547485 seconds
	be

	done in 4.7276611328125 seconds
	best classifier number 1551 with 1026 errors and weighed error 0.3909526321365849
	parity 1
	false positives so far: 879 or 0.9351063829787234

	round 36:
2940
	done in 4.804325342178345 seconds
	best classifier number 1544 with 1384 errors and weighed error 0.3927011565440654
	parity 1
	false positives so far: 877 or 0.9329787234042554

	round 37:
2940
	done in 4.920681953430176 seconds
	best classifier number 1069 with 1238 errors and weighed error 0.3758359265489957
	parity -1
	false positives so far: 863 or 0.9180851063829787

	round 38:
2940
	done in 4.940262794494629 seconds
	best classifier number 623 with 932 errors and weighed error 0.37153220046786717
	parity 1
	false positives so far: 851 or 0.9053191489361702

	round 39:
2940
	done in 4.8503899574279785 seconds
	best classifier number 798 with 615 errors and weighed error 0.38602400027356504
	parity -1
	false positives so far: 866 or 0.9212765957446809

	round 40:
2940
	done in 4.7626929283

	done in 4.881839275360107 seconds
	best classifier number 331 with 1512 errors and weighed error 0.39834351495872866
	parity -1
	false positives so far: 859 or 0.9138297872340425

	round 78:
2940
	done in 4.725288152694702 seconds
	best classifier number 395 with 1852 errors and weighed error 0.3899807361493966
	parity -1
	false positives so far: 868 or 0.9234042553191489

	round 79:
2940
	done in 4.92313814163208 seconds
	best classifier number 312 with 872 errors and weighed error 0.40034852491221606
	parity 1
	false positives so far: 905 or 0.9627659574468085

	round 80:
2940
	done in 4.772356986999512 seconds
	best classifier number 1365 with 1648 errors and weighed error 0.3799708096116313
	parity 1
	false positives so far: 910 or 0.9680851063829787



In [535]:
test_face_image_files = glob.glob(os.path.join(DataPath + 'test/FACES', '*.bmp'), recursive=True)

evaluation1 = 0

for i in range(1000):
    # import testing image
    bigimg = loadImg(test_face_image_files[i],IMG_SIZE,IMG_SIZE)

    iimg = integral(bigimg)

    stride = 2 
    increment = 2

    ftrs1 = feature1(iimg, IMG_SIZE, IMG_SIZE, stride, increment)
    ftrs2 = feature2(iimg, IMG_SIZE, IMG_SIZE, stride, increment)
    ftrs3 = feature3(iimg, IMG_SIZE, IMG_SIZE, stride, increment)
    ftrs4 = feature4(iimg, IMG_SIZE, IMG_SIZE, stride, increment)

    fts = ftrs1 + ftrs2 + ftrs3 + ftrs4 

    # evaluate whether there are faces in image
    evaluation1 += evaluate_cascade(fts, fa, fp, ft, fi)

print (evaluation1/1000)

0.994


In [536]:
test_nface_image_files = glob.glob(os.path.join(DataPath + 'test/NFACES', '*.bmp'), recursive=True)

evaluation2 = 0

for i in range(1000):
    # import testing image
    bigimg = loadImg(test_nface_image_files[i],IMG_SIZE,IMG_SIZE)

    iimg = integral(bigimg)

    stride = 2 
    increment = 2

    ftrs1 = feature1(iimg, IMG_SIZE, IMG_SIZE, stride, increment)
    ftrs2 = feature2(iimg, IMG_SIZE, IMG_SIZE, stride, increment)
    ftrs3 = feature3(iimg, IMG_SIZE, IMG_SIZE, stride, increment)
    ftrs4 = feature4(iimg, IMG_SIZE, IMG_SIZE, stride, increment)

    fts = ftrs1 + ftrs2 + ftrs3 + ftrs4 

    # evaluate whether there are faces in image
    evaluation2 += evaluate_cascade(fts, fa, fp, ft, fi)

print (evaluation2/1000)

0.387


In [537]:
featurelists = np.array(features)
(fc, fa, fp, ft, fi) = cascade(9, 80, featurelists, imagesv)

stage 0:
training with 1665 features over 4000 examples
	round 1:
4000
	done in 7.783377170562744 seconds
	best classifier number 1072 with 477 errors and weighed error 0.11925000000000845
	parity -1
	false positives so far: 2000 or 1.0

	round 2:
4000
	done in 7.458276987075806 seconds
	best classifier number 132 with 648 errors and weighed error 0.18803418803419314
	parity 1
	false positives so far: 1659 or 0.8295

	round 3:
4000
	done in 7.380171060562134 seconds
	best classifier number 1129 with 688 errors and weighed error 0.24671417820974234
	parity -1
	false positives so far: 1652 or 0.826

	round 4:
4000
	done in 6.92573094367981 seconds
	best classifier number 1644 with 1129 errors and weighed error 0.28193276508842147
	parity 1
	false positives so far: 1655 or 0.8275

	round 5:
4000
	done in 6.845652103424072 seconds
	best classifier number 955 with 888 errors and weighed error 0.30335224834872365
	parity 1
	false positives so far: 1369 or 0.6845

	round 6:
4000
	done in 6.84

	done in 6.757277965545654 seconds
	best classifier number 1069 with 1308 errors and weighed error 0.3942172263785378
	parity -1
	false positives so far: 1284 or 0.642

	round 47:
4000
	done in 6.885274171829224 seconds
	best classifier number 29 with 1259 errors and weighed error 0.3893137967136859
	parity 1
	false positives so far: 1315 or 0.6575

	round 48:
4000
	done in 6.910409927368164 seconds
	best classifier number 0 with 1449 errors and weighed error 0.3895490428533239
	parity 1
	false positives so far: 1309 or 0.6545

	round 49:
4000
	done in 6.896543264389038 seconds
	best classifier number 808 with 2637 errors and weighed error 0.3778535549965072
	parity -1
	false positives so far: 1306 or 0.653

	round 50:
4000
	done in 6.963835000991821 seconds
	best classifier number 1570 with 970 errors and weighed error 0.39574040907816677
	parity 1
	false positives so far: 1307 or 0.6535

	round 51:
4000
	done in 6.8919641971588135 seconds
	best classifier number 590 with 1444 errors 

	done in 5.884627103805542 seconds
	best classifier number 1376 with 787 errors and weighed error 0.3508620718450178
	parity 1
	false positives so far: 1025 or 0.73005698005698

	round 11:
3404
	done in 5.644200801849365 seconds
	best classifier number 267 with 994 errors and weighed error 0.3625066354568021
	parity -1
	false positives so far: 1077 or 0.7670940170940171

	round 12:
3404
	done in 5.635467052459717 seconds
	best classifier number 640 with 2024 errors and weighed error 0.3547309807175117
	parity -1
	false positives so far: 977 or 0.6958689458689459

	round 13:
3404
	done in 5.6839258670806885 seconds
	best classifier number 1236 with 852 errors and weighed error 0.36925835785109073
	parity -1
	false positives so far: 936 or 0.6666666666666666

	round 14:
3404
	done in 5.649264097213745 seconds
	best classifier number 1072 with 567 errors and weighed error 0.374416085002934
	parity -1
	false positives so far: 963 or 0.6858974358974359

	round 15:
3404
	done in 5.4961421489

	done in 28.85267400741577 seconds
	best classifier number 1126 with 1242 errors and weighed error 0.38628601471726126
	parity 1
	false positives so far: 1167 or 0.8311965811965812

	round 53:
3404
	done in 28.760537147521973 seconds
	best classifier number 1073 with 1315 errors and weighed error 0.39280646682584325
	parity -1
	false positives so far: 1166 or 0.8304843304843305

	round 54:
3404
	done in 27.959864139556885 seconds
	best classifier number 984 with 1833 errors and weighed error 0.38009023380610796
	parity 1
	false positives so far: 1143 or 0.8141025641025641

	round 55:
3404
	done in 22.94991421699524 seconds
	best classifier number 1307 with 1418 errors and weighed error 0.40764100853243873
	parity -1
	false positives so far: 1112 or 0.792022792022792

	round 56:
3404
	done in 22.50457215309143 seconds
	best classifier number 728 with 903 errors and weighed error 0.39663594125196744
	parity -1
	false positives so far: 1130 or 0.8048433048433048

	round 57:
3404
	done in 

	done in 10.211204051971436 seconds
	best classifier number 1078 with 1894 errors and weighed error 0.35179473703570124
	parity 1
	false positives so far: 942 or 0.8328912466843501

	round 15:
3131
	done in 10.145771980285645 seconds
	best classifier number 1067 with 1636 errors and weighed error 0.3495057661242725
	parity -1
	false positives so far: 994 or 0.878868258178603

	round 16:
3131
	done in 10.298249959945679 seconds
	best classifier number 135 with 1353 errors and weighed error 0.3676296468162115
	parity 1
	false positives so far: 1018 or 0.9000884173297966

	round 17:
3131
	done in 10.266412019729614 seconds
	best classifier number 96 with 563 errors and weighed error 0.3599458724864811
	parity 1
	false positives so far: 1037 or 0.9168877099911583

	round 18:
3131
	done in 10.283579349517822 seconds
	best classifier number 1073 with 1114 errors and weighed error 0.372468597638847
	parity -1
	false positives so far: 1034 or 0.9142351900972591

	round 19:
3131
	done in 10.240

...

	done in 4.644283056259155 seconds
	best classifier number 212 with 964 errors and weighed error 0.373716730857231
	parity 1
	false positives so far: 975 or 0.9989754098360656

	round 32:
2976
	done in 4.838354825973511 seconds
	best classifier number 1245 with 1278 errors and weighed error 0.3698610990466108
	parity 1
	false positives so far: 975 or 0.9989754098360656

	round 33:
2976
	done in 4.870326042175293 seconds
	best classifier number 1376 with 798 errors and weighed error 0.38261612351968716
	parity 1
	false positives so far: 975 or 0.9989754098360656

	round 34:
2976
	done in 4.758943796157837 seconds
	best classifier number 281 with 975 errors and weighed error 0.3837175943423802
	parity -1
	false positives so far: 975 or 0.9989754098360656

	round 35:
2976
	done in 4.714484930038452 seconds
	best classifier number 1103 with 563 errors and weighed error 0.38958577733949346
	parity -1
	false positives so far: 975 or 0.9989754098360656

	round 36:
2976
	done in 4.759910821914

	done in 4.6592772006988525 seconds
	best classifier number 1035 with 1413 errors and weighed error 0.3997141570307392
	parity -1
	false positives so far: 915 or 0.9375

	round 74:
2976
	done in 4.7525787353515625 seconds
	best classifier number 0 with 851 errors and weighed error 0.40817609010662964
	parity 1
	false positives so far: 940 or 0.9631147540983607

	round 75:
2976
	done in 4.660758018493652 seconds
	best classifier number 182 with 1394 errors and weighed error 0.405356966513906
	parity -1
	false positives so far: 940 or 0.9631147540983607

	round 76:
2976
	done in 4.891417741775513 seconds
	best classifier number 79 with 701 errors and weighed error 0.4013175973810257
	parity 1
	false positives so far: 946 or 0.9692622950819673

	round 77:
2976
	done in 4.810495853424072 seconds
	best classifier number 1615 with 1049 errors and weighed error 0.3888822760832363
	parity 1
	false positives so far: 943 or 0.9661885245901639

	round 78:
2976
	done in 4.829486846923828 seconds
	

	done in 4.957555294036865 seconds
	best classifier number 1551 with 1026 errors and weighed error 0.3909526321365849
	parity 1
	false positives so far: 879 or 0.9351063829787234

	round 36:
2940
	done in 4.927695989608765 seconds
	best classifier number 1544 with 1384 errors and weighed error 0.3927011565440654
	parity 1
	false positives so far: 877 or 0.9329787234042554

	round 37:
2940
	done in 4.7531702518463135 seconds
	best classifier number 1069 with 1238 errors and weighed error 0.3758359265489957
	parity -1
	false positives so far: 863 or 0.9180851063829787

	round 38:
2940
	done in 4.724133253097534 seconds
	best classifier number 623 with 932 errors and weighed error 0.37153220046786717
	parity 1
	false positives so far: 851 or 0.9053191489361702

	round 39:
2940
	done in 4.718657970428467 seconds
	best classifier number 798 with 615 errors and weighed error 0.38602400027356504
	parity -1
	false positives so far: 866 or 0.9212765957446809

	round 40:
2940
	done in 4.79808020

	done in 4.791216135025024 seconds
	best classifier number 331 with 1512 errors and weighed error 0.39834351495872866
	parity -1
	false positives so far: 859 or 0.9138297872340425

	round 78:
2940
	done in 4.652900218963623 seconds
	best classifier number 395 with 1852 errors and weighed error 0.3899807361493966
	parity -1
	false positives so far: 868 or 0.9234042553191489

	round 79:
2940
	done in 4.693341970443726 seconds
	best classifier number 312 with 872 errors and weighed error 0.40034852491221606
	parity 1
	false positives so far: 905 or 0.9627659574468085

	round 80:
2940
	done in 4.673655033111572 seconds
	best classifier number 1365 with 1648 errors and weighed error 0.3799708096116313
	parity 1
	false positives so far: 910 or 0.9680851063829787

stage 8:
training with 1665 features over 2910 examples
	round 1:
2910
	done in 5.212607145309448 seconds
	best classifier number 1072 with 440 errors and weighed error 0.16270329670330175
	parity -1
	false positives so far: 910 or 

	done in 4.640980958938599 seconds
	best classifier number 29 with 733 errors and weighed error 0.38991695879672783
	parity 1
	false positives so far: 869 or 0.954945054945055

	round 40:
2910
	done in 4.634221792221069 seconds
	best classifier number 395 with 1838 errors and weighed error 0.387913715550401
	parity -1
	false positives so far: 866 or 0.9516483516483516

	round 41:
2910
	done in 4.5922229290008545 seconds
	best classifier number 1656 with 1023 errors and weighed error 0.38701221104292305
	parity -1
	false positives so far: 886 or 0.9736263736263736

	round 42:
2910
	done in 4.545325994491577 seconds
	best classifier number 1034 with 1974 errors and weighed error 0.3922557265127832
	parity -1
	false positives so far: 892 or 0.9802197802197802

	round 43:
2910
	done in 4.7192792892456055 seconds
	best classifier number 590 with 1112 errors and weighed error 0.3936042944648056
	parity 1
	false positives so far: 879 or 0.9659340659340659

	round 44:
2910
	done in 4.811376094

In [538]:
test_face_image_files = glob.glob(os.path.join(DataPath + 'test/FACES', '*.bmp'), recursive=True)

evaluation1 = 0

for i in range(1000):
    # import testing image
    bigimg = loadImg(test_face_image_files[i],IMG_SIZE,IMG_SIZE)

    iimg = integral(bigimg)

    stride = 2 
    increment = 2

    ftrs1 = feature1(iimg, IMG_SIZE, IMG_SIZE, stride, increment)
    ftrs2 = feature2(iimg, IMG_SIZE, IMG_SIZE, stride, increment)
    ftrs3 = feature3(iimg, IMG_SIZE, IMG_SIZE, stride, increment)
    ftrs4 = feature4(iimg, IMG_SIZE, IMG_SIZE, stride, increment)

    fts = ftrs1 + ftrs2 + ftrs3 + ftrs4 

    # evaluate whether there are faces in image
    evaluation1 += evaluate_cascade(fts, fa, fp, ft, fi)

print (evaluation1/1000)

0.994


In [539]:
test_nface_image_files = glob.glob(os.path.join(DataPath + 'test/NFACES', '*.bmp'), recursive=True)

evaluation2 = 0

for i in range(1000):
    # import testing image
    bigimg = loadImg(test_nface_image_files[i],IMG_SIZE,IMG_SIZE)

    iimg = integral(bigimg)

    stride = 2 
    increment = 2

    ftrs1 = feature1(iimg, IMG_SIZE, IMG_SIZE, stride, increment)
    ftrs2 = feature2(iimg, IMG_SIZE, IMG_SIZE, stride, increment)
    ftrs3 = feature3(iimg, IMG_SIZE, IMG_SIZE, stride, increment)
    ftrs4 = feature4(iimg, IMG_SIZE, IMG_SIZE, stride, increment)

    fts = ftrs1 + ftrs2 + ftrs3 + ftrs4 

    # evaluate whether there are faces in image
    evaluation2 += evaluate_cascade(fts, fa, fp, ft, fi)

print (evaluation2/1000)

0.385


In [540]:
featurelists = np.array(features)
(fc, fa, fp, ft, fi) = cascade(10, 80, featurelists, imagesv)

stage 0:
training with 1665 features over 4000 examples
	round 1:
4000
	done in 7.215140104293823 seconds
	best classifier number 1072 with 477 errors and weighed error 0.11925000000000845
	parity -1
	false positives so far: 2000 or 1.0

	round 2:
4000
	done in 6.8610999584198 seconds
	best classifier number 132 with 648 errors and weighed error 0.18803418803419314
	parity 1
	false positives so far: 1659 or 0.8295

	round 3:
4000
	done in 7.0191969871521 seconds
	best classifier number 1129 with 688 errors and weighed error 0.24671417820974234
	parity -1
	false positives so far: 1652 or 0.826

	round 4:
4000
	done in 6.547688007354736 seconds
	best classifier number 1644 with 1129 errors and weighed error 0.28193276508842147
	parity 1
	false positives so far: 1655 or 0.8275

	round 5:
4000
	done in 6.934308767318726 seconds
	best classifier number 955 with 888 errors and weighed error 0.30335224834872365
	parity 1
	false positives so far: 1369 or 0.6845

	round 6:
4000
	done in 7.07220

	done in 6.893458127975464 seconds
	best classifier number 1069 with 1308 errors and weighed error 0.3942172263785378
	parity -1
	false positives so far: 1284 or 0.642

	round 47:
4000
	done in 7.088599920272827 seconds
	best classifier number 29 with 1259 errors and weighed error 0.3893137967136859
	parity 1
	false positives so far: 1315 or 0.6575

	round 48:
4000
	done in 6.797716856002808 seconds
	best classifier number 0 with 1449 errors and weighed error 0.3895490428533239
	parity 1
	false positives so far: 1309 or 0.6545

	round 49:
4000
	done in 6.8953258991241455 seconds
	best classifier number 808 with 2637 errors and weighed error 0.3778535549965072
	parity -1
	false positives so far: 1306 or 0.653

	round 50:
4000
	done in 6.7665791511535645 seconds
	best classifier number 1570 with 970 errors and weighed error 0.39574040907816677
	parity 1
	false positives so far: 1307 or 0.6535

	round 51:
4000
	done in 6.624698877334595 seconds
	best classifier number 590 with 1444 errors

	done in 5.346611976623535 seconds
	best classifier number 1376 with 787 errors and weighed error 0.3508620718450178
	parity 1
	false positives so far: 1025 or 0.73005698005698

	round 11:
3404
	done in 5.463217735290527 seconds
	best classifier number 267 with 994 errors and weighed error 0.3625066354568021
	parity -1
	false positives so far: 1077 or 0.7670940170940171

	round 12:
3404
	done in 5.475656986236572 seconds
	best classifier number 640 with 2024 errors and weighed error 0.3547309807175117
	parity -1
	false positives so far: 977 or 0.6958689458689459

	round 13:
3404
	done in 5.5599071979522705 seconds
	best classifier number 1236 with 852 errors and weighed error 0.36925835785109073
	parity -1
	false positives so far: 936 or 0.6666666666666666

	round 14:
3404
	done in 5.537948846817017 seconds
	best classifier number 1072 with 567 errors and weighed error 0.374416085002934
	parity -1
	false positives so far: 963 or 0.6858974358974359

	round 15:
3404
	done in 5.4400920867

	done in 5.493226051330566 seconds
	best classifier number 1126 with 1242 errors and weighed error 0.38628601471726126
	parity 1
	false positives so far: 1167 or 0.8311965811965812

	round 53:
3404
	done in 5.5248730182647705 seconds
	best classifier number 1073 with 1315 errors and weighed error 0.39280646682584325
	parity -1
	false positives so far: 1166 or 0.8304843304843305

	round 54:
3404
	done in 5.740487813949585 seconds
	best classifier number 984 with 1833 errors and weighed error 0.38009023380610796
	parity 1
	false positives so far: 1143 or 0.8141025641025641

	round 55:
3404
	done in 5.420292854309082 seconds
	best classifier number 1307 with 1418 errors and weighed error 0.40764100853243873
	parity -1
	false positives so far: 1112 or 0.792022792022792

	round 56:
3404
	done in 5.583081960678101 seconds
	best classifier number 728 with 903 errors and weighed error 0.39663594125196744
	parity -1
	false positives so far: 1130 or 0.8048433048433048

	round 57:
3404
	done in 5

	done in 5.150192022323608 seconds
	best classifier number 1078 with 1894 errors and weighed error 0.35179473703570124
	parity 1
	false positives so far: 942 or 0.8328912466843501

	round 15:
3131
	done in 5.276088714599609 seconds
	best classifier number 1067 with 1636 errors and weighed error 0.3495057661242725
	parity -1
	false positives so far: 994 or 0.878868258178603

	round 16:
3131
	done in 5.452094316482544 seconds
	best classifier number 135 with 1353 errors and weighed error 0.3676296468162115
	parity 1
	false positives so far: 1018 or 0.9000884173297966

	round 17:
3131
	done in 5.1174821853637695 seconds
	best classifier number 96 with 563 errors and weighed error 0.3599458724864811
	parity 1
	false positives so far: 1037 or 0.9168877099911583

	round 18:
3131
	done in 5.317367076873779 seconds
	best classifier number 1073 with 1114 errors and weighed error 0.372468597638847
	parity -1
	false positives so far: 1034 or 0.9142351900972591

	round 19:
3131
	done in 5.23133587

...

	done in 4.826333045959473 seconds
	best classifier number 1544 with 1384 errors and weighed error 0.3927011565440654
	parity 1
	false positives so far: 877 or 0.9329787234042554

	round 37:
2940
	done in 4.728543996810913 seconds
	best classifier number 1069 with 1238 errors and weighed error 0.3758359265489957
	parity -1
	false positives so far: 863 or 0.9180851063829787

	round 38:
2940
	done in 4.699625015258789 seconds
	best classifier number 623 with 932 errors and weighed error 0.37153220046786717
	parity 1
	false positives so far: 851 or 0.9053191489361702

	round 39:
2940
	done in 4.63834810256958 seconds
	best classifier number 798 with 615 errors and weighed error 0.38602400027356504
	parity -1
	false positives so far: 866 or 0.9212765957446809

	round 40:
2940
	done in 4.767492055892944 seconds
	best classifier number 1034 with 2002 errors and weighed error 0.36596340380337095
	parity -1
	false positives so far: 853 or 0.9074468085106383

	round 41:
2940
	done in 4.69047284

	done in 4.813536882400513 seconds
	best classifier number 395 with 1852 errors and weighed error 0.3899807361493966
	parity -1
	false positives so far: 868 or 0.9234042553191489

	round 79:
2940
	done in 4.708170175552368 seconds
	best classifier number 312 with 872 errors and weighed error 0.40034852491221606
	parity 1
	false positives so far: 905 or 0.9627659574468085

	round 80:
2940
	done in 4.8102099895477295 seconds
	best classifier number 1365 with 1648 errors and weighed error 0.3799708096116313
	parity 1
	false positives so far: 910 or 0.9680851063829787

stage 8:
training with 1665 features over 2910 examples
	round 1:
2910
	done in 5.323779106140137 seconds
	best classifier number 1072 with 440 errors and weighed error 0.16270329670330175
	parity -1
	false positives so far: 910 or 1.0

	round 2:
2910
	done in 4.775237798690796 seconds
	best classifier number 133 with 507 errors and weighed error 0.2138034670494283
	parity 1
	false positives so far: 837 or 0.9197802197802197

	done in 4.928884983062744 seconds
	best classifier number 395 with 1838 errors and weighed error 0.387913715550401
	parity -1
	false positives so far: 866 or 0.9516483516483516

	round 41:
2910
	done in 4.686598777770996 seconds
	best classifier number 1656 with 1023 errors and weighed error 0.38701221104292305
	parity -1
	false positives so far: 886 or 0.9736263736263736

	round 42:
2910
	done in 4.605774164199829 seconds
	best classifier number 1034 with 1974 errors and weighed error 0.3922557265127832
	parity -1
	false positives so far: 892 or 0.9802197802197802

	round 43:
2910
	done in 4.83602499961853 seconds
	best classifier number 590 with 1112 errors and weighed error 0.3936042944648056
	parity 1
	false positives so far: 879 or 0.9659340659340659

	round 44:
2910
	done in 4.8544886112213135 seconds
	best classifier number 318 with 848 errors and weighed error 0.3906671675449743
	parity 1
	false positives so far: 878 or 0.9648351648351648

	round 45:
2910
	done in 4.7018170356

	done in 4.955162763595581 seconds
	best classifier number 133 with 506 errors and weighed error 0.21404982175930032
	parity 1
	false positives so far: 834 or 0.9276974416017798

	round 3:
2899
	done in 4.786057233810425 seconds
	best classifier number 1129 with 736 errors and weighed error 0.29263680012241977
	parity -1
	false positives so far: 835 or 0.92880978865406

	round 4:
2899
	done in 4.897839069366455 seconds
	best classifier number 1644 with 955 errors and weighed error 0.31197300519736565
	parity 1
	false positives so far: 835 or 0.92880978865406

	round 5:
2899
	done in 4.9513609409332275 seconds
	best classifier number 855 with 664 errors and weighed error 0.31756355506895445
	parity 1
	false positives so far: 789 or 0.8776418242491657

	round 6:
2899
	done in 4.785372972488403 seconds
	best classifier number 175 with 1443 errors and weighed error 0.33341558533313354
	parity 1
	false positives so far: 825 or 0.917686318131257

	round 7:
2899
	done in 4.724219083786011 sec

	done in 4.69752311706543 seconds
	best classifier number 1126 with 798 errors and weighed error 0.37467892922990714
	parity 1
	false positives so far: 807 or 0.8976640711902113

	round 45:
2899
	done in 4.970523118972778 seconds
	best classifier number 1037 with 1183 errors and weighed error 0.3931097692425647
	parity -1
	false positives so far: 808 or 0.8987764182424917

	round 46:
2899
	done in 4.992227077484131 seconds
	best classifier number 984 with 1500 errors and weighed error 0.3923478895930633
	parity 1
	false positives so far: 813 or 0.9043381535038932

	round 47:
2899
	done in 5.062171220779419 seconds
	best classifier number 76 with 504 errors and weighed error 0.39545874349354365
	parity 1
	false positives so far: 835 or 0.92880978865406

	round 48:
2899
	done in 4.88963508605957 seconds
	best classifier number 1461 with 1850 errors and weighed error 0.390825367839518
	parity 1
	false positives so far: 836 or 0.9299221357063404

	round 49:
2899
	done in 4.961852788925171 

In [541]:
test_face_image_files = glob.glob(os.path.join(DataPath + 'test/FACES', '*.bmp'), recursive=True)

evaluation1 = 0

for i in range(1000):
    # import testing image
    bigimg = loadImg(test_face_image_files[i],IMG_SIZE,IMG_SIZE)

    iimg = integral(bigimg)

    stride = 2 
    increment = 2

    ftrs1 = feature1(iimg, IMG_SIZE, IMG_SIZE, stride, increment)
    ftrs2 = feature2(iimg, IMG_SIZE, IMG_SIZE, stride, increment)
    ftrs3 = feature3(iimg, IMG_SIZE, IMG_SIZE, stride, increment)
    ftrs4 = feature4(iimg, IMG_SIZE, IMG_SIZE, stride, increment)

    fts = ftrs1 + ftrs2 + ftrs3 + ftrs4 

    # evaluate whether there are faces in image
    evaluation1 += evaluate_cascade(fts, fa, fp, ft, fi)

print (evaluation1/1000)

0.994


In [542]:
test_nface_image_files = glob.glob(os.path.join(DataPath + 'test/NFACES', '*.bmp'), recursive=True)

evaluation2 = 0

for i in range(1000):
    # import testing image
    bigimg = loadImg(test_nface_image_files[i],IMG_SIZE,IMG_SIZE)

    iimg = integral(bigimg)

    stride = 2 
    increment = 2

    ftrs1 = feature1(iimg, IMG_SIZE, IMG_SIZE, stride, increment)
    ftrs2 = feature2(iimg, IMG_SIZE, IMG_SIZE, stride, increment)
    ftrs3 = feature3(iimg, IMG_SIZE, IMG_SIZE, stride, increment)
    ftrs4 = feature4(iimg, IMG_SIZE, IMG_SIZE, stride, increment)

    fts = ftrs1 + ftrs2 + ftrs3 + ftrs4 

    # evaluate whether there are faces in image
    evaluation2 += evaluate_cascade(fts, fa, fp, ft, fi)

print (evaluation2/1000)

0.375
